In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns
from scipy.stats import f_oneway

In [4]:
def run_device_regression(dataframe, keyword=None, target_column='ConversionRate', baseline_device=None):
    # Filter by keyword only if keyword is specified
    if keyword is not None:
        subset = dataframe[dataframe['Keyword'] == keyword]
    else:
        subset = dataframe

    # Reorder categories so the baseline device is first
    if baseline_device is not None and baseline_device in subset['Device'].unique():
        subset['Device'] = pd.Categorical(
            subset['Device'],
            categories=[baseline_device] + [d for d in subset['Device'].unique() if d != baseline_device],
            ordered=True
        )
    elif baseline_device is not None:
        print(f"Warning: baseline device '{baseline_device}' not found in subset.")

    # One-hot encode devices, dropping the baseline
    X = pd.get_dummies(subset['Device'], drop_first=True)
    X = sm.add_constant(X)
    X = X.astype(float)

    # Target variable
    y = subset[target_column].astype(float)

    # Run regression
    model = sm.OLS(y, X).fit()

    # Print summary
    title = f"Keyword: '{keyword}'" if keyword else "All Keywords"
    print(f"\n--- Regression Summary for {title} ---\n")
    print(model.summary())

    return model

In [3]:
def run_device_diff_regression(dataframe, target_column, baseline_device=None):
    # Sort the dataframe by Device and Ad_Date to ensure proper ordering
    dataframe = dataframe.sort_values(by=['Device', 'Ad_Date'])

    # Compute difference of the target variable within each Device group
    dataframe['Target_Diff'] = dataframe.groupby('Device', observed=True)[target_column].diff(-1)

    # Drop rows with NaNs caused by diff, and make a copy to avoid SettingWithCopyWarning
    subset = dataframe.dropna(subset=['Target_Diff']).copy()

    # Set baseline device if specified
    if baseline_device and baseline_device in subset['Device'].unique():
        subset['Device'] = pd.Categorical(
            subset['Device'],
            categories=[baseline_device] + [d for d in subset['Device'].unique() if d != baseline_device],
            ordered=True
        )
    elif baseline_device:
        print(f"Warning: baseline device '{baseline_device}' not found in data.")

    # Dummy encode Device (drop first category = baseline)
    X = pd.get_dummies(subset['Device'], drop_first=True)
    X = sm.add_constant(X)
    X = X.astype(float)

    # Response variable is the difference
    y = subset['Target_Diff'].astype(float)

    # Run regression
    model = sm.OLS(y, X).fit()

    print(f"\n--- Regression Summary on Differences in '{target_column}' by Device ---\n")
    print(model.summary())
    
    return model

In [2]:
device_rollups= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\device_rollups.csv", delimiter=';', quotechar='"',
                            engine= 'python')
device_rollups

,Ad_Date,Device,daily_spend,rolling_cost,daily_clicks,r_clicks,daily_impressions,r_impressions,daily_leads,r_leads,daily_conversions,r_conversions,daily_sales,r_sales,avg_conversions,weighted_conversion_rate
0,2024-11-01,desktop,6117.86,6117.86,3851.0,3851.0,129174.0,129174.0,531.0,531.0,189.0,189.0,37978.0,37978.0,0.051800,0.049078
1,2024-11-02,desktop,4739.13,10856.99,3202.0,7053.0,113790.0,242964.0,463.0,994.0,155.0,344.0,33619.0,71597.0,0.048955,0.048407
2,2024-11-03,desktop,6292.33,17149.32,4163.0,11216.0,141903.0,384867.0,677.0,1671.0,199.0,543.0,42029.0,113626.0,0.051458,0.047802
3,2024-11-04,desktop,6380.53,23529.85,4180.0,15396.0,127917.0,512784.0,596.0,2267.0,205.0,748.0,43494.0,157120.0,0.055720,0.049043
4,2024-11-05,desktop,4727.63,28257.48,3730.0,19126.0,103677.0,616461.0,557.0,2824.0,160.0,908.0,35675.0,192795.0,0.046273,0.042895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2024-11-26,tablet,6681.37,133412.25,4393.0,89607.0,147346.0,2921555.0,624.0,13103.0,214.0,4131.0,49608.0,918446.0,0.051692,0.048714
86,2024-11-27,tablet,5446.57,138858.82,3397.0,93004.0,115806.0,3037361.0,437.0,13540.0,159.0,4290.0,34004.0,952450.0,0.051182,0.046806
87,2024-11-28,tablet,4494.38,143353.20,3436.0,96440.0,98067.0,3135428.0,434.0,13974.0,154.0,4444.0,32335.0,984785.0,0.045526,0.044820
88,2024-11-29,tablet,4177.14,147530.34,2888.0,99328.0,102572.0,3238000.0,385.0,14359.0,131.0,4575.0,29652.0,1014437.0,0.048176,0.045360


In [6]:
d_r_averages= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\device_rollup_averages.csv", delimiter=';', quotechar='"',
                            engine= 'python')
d_r_averages

,Device,avg_daily_spend,spend_volatility,avg_daily_clicks,clicks_volatility,avg_daily_impressions,impressions_volatility,avg_daily_leads,leads_volatility,avg_daily_conversions,conversions_volatility,avg_daily_sales,sales_volatility
0,desktop,5446.61,1107.24,3621.43,727.75,119010.70,25146.77,522.47,103.07,171.87,32.40,37385.93,7166.48
1,mobile,5563.39,1038.94,3724.57,693.77,120813.70,23673.61,533.33,110.76,175.03,34.32,37526.57,7315.58
2,tablet,5088.04,1220.59,3439.30,794.49,111784.57,25373.49,495.47,116.50,157.70,37.56,35088.37,8022.74


In [ ]:
#running tests using highest and lowest in each num col as rotating constants

In [5]:
run_device_regression(device_rollups, target_column='daily_spend', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.407
Date:                Sun, 21 Sep 2025   Prob (F-statistic):              0.250
Time:                        18:15:50   Log-Likelihood:                -759.98
No. Observations:                  90   AIC:                             1526.
Df Residuals:                      87   BIC:                             1533.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [35]:
print("\nMean and variance of target by device:")
print(device_rollups.groupby('Device')['daily_spend'].agg(['mean', 'std', 'count']))


Mean and variance of target by device:
                mean          std  count
Device                                  
tablet   5088.040000  1241.459508     30
desktop  5446.610333  1126.165377     30
mobile   5563.393667  1056.702839     30


C:\Users\nehir\AppData\Local\Temp\ipykernel_1160\3858946394.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(device_rollups.groupby('Device')['daily_spend'].agg(['mean', 'std', 'count']))


In [10]:
run_device_regression(device_rollups, target_column='daily_spend', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.407
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.250
Time:                        18:54:39   Log-Likelihood:                -759.98
No. Observations:                  90   AIC:                             1526.
Df Residuals:                      87   BIC:                             1533.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [11]:
run_device_regression(device_rollups, target_column='daily_clicks', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.105
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.336
Time:                        18:56:10   Log-Likelihood:                -722.28
No. Observations:                  90   AIC:                             1451.
Df Residuals:                      87   BIC:                             1458.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [12]:
run_device_regression(device_rollups, target_column='daily_clicks', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.105
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.336
Time:                        18:56:21   Log-Likelihood:                -722.28
No. Observations:                  90   AIC:                             1451.
Df Residuals:                      87   BIC:                             1458.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [13]:
run_device_regression(device_rollups, target_column='daily_impressions', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.082
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.344
Time:                        18:56:50   Log-Likelihood:                -1038.2
No. Observations:                  90   AIC:                             2082.
Df Residuals:                      87   BIC:                             2090.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [14]:
run_device_regression(device_rollups, target_column='daily_impressions', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.082
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.344
Time:                        18:57:01   Log-Likelihood:                -1038.2
No. Observations:                  90   AIC:                             2082.
Df Residuals:                      87   BIC:                             2090.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [15]:
run_device_regression(device_rollups, target_column='daily_leads', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9071
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.407
Time:                        19:01:49   Log-Likelihood:                -550.95
No. Observations:                  90   AIC:                             1108.
Df Residuals:                      87   BIC:                             1115.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [16]:
run_device_regression(device_rollups, target_column='daily_leads', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9071
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.407
Time:                        19:02:00   Log-Likelihood:                -550.95
No. Observations:                  90   AIC:                             1108.
Df Residuals:                      87   BIC:                             1115.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [17]:
run_device_regression(device_rollups, target_column='daily_conversions', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     2.037
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.137
Time:                        19:02:29   Log-Likelihood:                -447.24
No. Observations:                  90   AIC:                             900.5
Df Residuals:                      87   BIC:                             908.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [18]:
run_device_regression(device_rollups, target_column='daily_conversions', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     2.037
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.137
Time:                        19:02:42   Log-Likelihood:                -447.24
No. Observations:                  90   AIC:                             900.5
Df Residuals:                      87   BIC:                             908.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [19]:
run_device_regression(device_rollups, target_column='daily_sales', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9633
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.386
Time:                        19:03:58   Log-Likelihood:                -930.87
No. Observations:                  90   AIC:                             1868.
Df Residuals:                      87   BIC:                             1875.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [20]:
run_device_regression(device_rollups, target_column='daily_sales', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9633
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.386
Time:                        19:04:08   Log-Likelihood:                -930.87
No. Observations:                  90   AIC:                             1868.
Df Residuals:                      87   BIC:                             1875.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [21]:
run_device_regression(device_rollups, target_column='avg_conversions', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:        avg_conversions   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.2901
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.749
Time:                        19:04:39   Log-Likelihood:                 352.98
No. Observations:                  90   AIC:                            -700.0
Df Residuals:                      87   BIC:                            -692.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [22]:
run_device_regression(device_rollups, target_column='avg_conversions', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:        avg_conversions   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.2901
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.749
Time:                        19:04:49   Log-Likelihood:                 352.98
No. Observations:                  90   AIC:                            -700.0
Df Residuals:                      87   BIC:                            -692.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [23]:
run_device_regression(device_rollups, target_column='weighted_conversion_rate', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                               OLS Regression Results                               
Dep. Variable:     weighted_conversion_rate   R-squared:                       0.033
Model:                                  OLS   Adj. R-squared:                  0.010
Method:                       Least Squares   F-statistic:                     1.467
Date:                      Tue, 09 Sep 2025   Prob (F-statistic):              0.236
Time:                              19:05:57   Log-Likelihood:                 364.95
No. Observations:                        90   AIC:                            -723.9
Df Residuals:                            87   BIC:                            -716.4
Df Model:                                 2                                         
Covariance Type:                  nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------

In [24]:
run_device_regression(device_rollups, target_column='weighted_conversion_rate', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                               OLS Regression Results                               
Dep. Variable:     weighted_conversion_rate   R-squared:                       0.033
Model:                                  OLS   Adj. R-squared:                  0.010
Method:                       Least Squares   F-statistic:                     1.467
Date:                      Tue, 09 Sep 2025   Prob (F-statistic):              0.236
Time:                              19:06:06   Log-Likelihood:                 364.95
No. Observations:                        90   AIC:                            -723.9
Df Residuals:                            87   BIC:                            -716.4
Df Model:                                 2                                         
Covariance Type:                  nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------

In [ ]:
# doing the same for volatility on day-to-day totals

In [26]:
run_device_diff_regression(device_rollups, target_column='daily_spend', baseline_device='tablet')


--- Regression Summary on Differences in 'daily_spend' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02872
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.972
Time:                        19:08:48   Log-Likelihood:                -763.18
No. Observations:                  87   AIC:                             1532.
Df Residuals:                      84   BIC:                             1540.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

In [25]:
run_device_diff_regression(device_rollups, target_column='daily_spend', baseline_device='mobile')


--- Regression Summary on Differences in 'daily_spend' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02872
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.972
Time:                        19:08:44   Log-Likelihood:                -763.18
No. Observations:                  87   AIC:                             1532.
Df Residuals:                      84   BIC:                             1540.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

In [28]:
run_device_diff_regression(device_rollups, target_column='daily_clicks', baseline_device='mobile')


--- Regression Summary on Differences in 'daily_clicks' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02630
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.974
Time:                        19:09:33   Log-Likelihood:                -727.66
No. Observations:                  87   AIC:                             1461.
Df Residuals:                      84   BIC:                             1469.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

In [27]:
run_device_diff_regression(device_rollups, target_column='daily_clicks', baseline_device='tablet')


--- Regression Summary on Differences in 'daily_clicks' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02630
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.974
Time:                        19:09:29   Log-Likelihood:                -727.66
No. Observations:                  87   AIC:                             1461.
Df Residuals:                      84   BIC:                             1469.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

In [30]:
run_device_diff_regression(device_rollups, target_column='daily_impressions', baseline_device='mobile')


--- Regression Summary on Differences in 'daily_impressions' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.03201
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.969
Time:                        19:10:24   Log-Likelihood:                -1033.7
No. Observations:                  87   AIC:                             2073.
Df Residuals:                      84   BIC:                             2081.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [29]:
run_device_diff_regression(device_rollups, target_column='daily_impressions', baseline_device='tablet')


--- Regression Summary on Differences in 'daily_impressions' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.03201
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.969
Time:                        19:10:20   Log-Likelihood:                -1033.7
No. Observations:                  87   AIC:                             2073.
Df Residuals:                      84   BIC:                             2081.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [32]:
run_device_diff_regression(device_rollups, target_column='daily_leads', baseline_device='desktop')


--- Regression Summary on Differences in 'daily_leads' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02393
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.976
Time:                        19:11:57   Log-Likelihood:                -558.47
No. Observations:                  87   AIC:                             1123.
Df Residuals:                      84   BIC:                             1130.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

In [31]:
run_device_diff_regression(device_rollups, target_column='daily_leads', baseline_device='tablet')


--- Regression Summary on Differences in 'daily_leads' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02393
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.976
Time:                        19:11:45   Log-Likelihood:                -558.47
No. Observations:                  87   AIC:                             1123.
Df Residuals:                      84   BIC:                             1130.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

In [37]:
run_device_diff_regression(device_rollups, target_column='daily_conversions', baseline_device='tablet')


--- Regression Summary on Differences in 'daily_conversions' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02562
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.975
Time:                        19:48:26   Log-Likelihood:                -461.20
No. Observations:                  87   AIC:                             928.4
Df Residuals:                      84   BIC:                             935.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [36]:
run_device_diff_regression(device_rollups, target_column='daily_conversions', baseline_device='desktop')


--- Regression Summary on Differences in 'daily_conversions' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02562
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.975
Time:                        19:48:21   Log-Likelihood:                -461.20
No. Observations:                  87   AIC:                             928.4
Df Residuals:                      84   BIC:                             935.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [39]:
run_device_diff_regression(device_rollups, target_column='daily_sales', baseline_device='desktop')


--- Regression Summary on Differences in 'daily_sales' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02514
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.975
Time:                        19:49:17   Log-Likelihood:                -930.62
No. Observations:                  87   AIC:                             1867.
Df Residuals:                      84   BIC:                             1875.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

In [38]:
run_device_diff_regression(device_rollups, target_column='daily_sales', baseline_device='tablet')


--- Regression Summary on Differences in 'daily_sales' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02514
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.975
Time:                        19:49:13   Log-Likelihood:                -930.62
No. Observations:                  87   AIC:                             1867.
Df Residuals:                      84   BIC:                             1875.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

In [40]:
run_device_diff_regression(device_rollups, target_column='rolling_cost', baseline_device='tablet')


--- Regression Summary on Differences in 'rolling_cost' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.500
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.229
Time:                        19:51:48   Log-Likelihood:                -734.63
No. Observations:                  87   AIC:                             1475.
Df Residuals:                      84   BIC:                             1483.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

In [41]:
run_device_diff_regression(device_rollups, target_column='rolling_cost', baseline_device='mobile')


--- Regression Summary on Differences in 'rolling_cost' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.500
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.229
Time:                        19:51:51   Log-Likelihood:                -734.63
No. Observations:                  87   AIC:                             1475.
Df Residuals:                      84   BIC:                             1483.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

In [42]:
run_device_diff_regression(device_rollups, target_column='r_clicks', baseline_device='mobile')


--- Regression Summary on Differences in 'r_clicks' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.272
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.286
Time:                        19:51:55   Log-Likelihood:                -698.15
No. Observations:                  87   AIC:                             1402.
Df Residuals:                      84   BIC:                             1410.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [43]:
run_device_diff_regression(device_rollups, target_column='r_clicks', baseline_device='tablet')


--- Regression Summary on Differences in 'r_clicks' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.272
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.286
Time:                        19:51:57   Log-Likelihood:                -698.15
No. Observations:                  87   AIC:                             1402.
Df Residuals:                      84   BIC:                             1410.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [44]:
run_device_diff_regression(device_rollups, target_column='r_impressions', baseline_device='mobile')


--- Regression Summary on Differences in 'r_impressions' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.239
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.295
Time:                        19:51:58   Log-Likelihood:                -1002.5
No. Observations:                  87   AIC:                             2011.
Df Residuals:                      84   BIC:                             2018.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [45]:
run_device_diff_regression(device_rollups, target_column='r_impressions', baseline_device='tablet')


--- Regression Summary on Differences in 'r_impressions' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.239
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.295
Time:                        19:52:01   Log-Likelihood:                -1002.5
No. Observations:                  87   AIC:                             2011.
Df Residuals:                      84   BIC:                             2018.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [46]:
run_device_diff_regression(device_rollups, target_column='r_leads', baseline_device='desktop')


--- Regression Summary on Differences in 'r_leads' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.058
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.352
Time:                        19:52:04   Log-Likelihood:                -532.70
No. Observations:                  87   AIC:                             1071.
Df Residuals:                      84   BIC:                             1079.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

In [47]:
run_device_diff_regression(device_rollups, target_column='r_leads', baseline_device='tablet')


--- Regression Summary on Differences in 'r_leads' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.058
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.352
Time:                        19:52:05   Log-Likelihood:                -532.70
No. Observations:                  87   AIC:                             1071.
Df Residuals:                      84   BIC:                             1079.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

In [48]:
run_device_diff_regression(device_rollups, target_column='r_conversions', baseline_device='tablet')


--- Regression Summary on Differences in 'r_conversions' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     2.111
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.127
Time:                        19:52:06   Log-Likelihood:                -431.70
No. Observations:                  87   AIC:                             869.4
Df Residuals:                      84   BIC:                             876.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [49]:
run_device_diff_regression(device_rollups, target_column='r_conversions', baseline_device='desktop')


--- Regression Summary on Differences in 'r_conversions' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     2.111
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.127
Time:                        19:52:07   Log-Likelihood:                -431.70
No. Observations:                  87   AIC:                             869.4
Df Residuals:                      84   BIC:                             876.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [50]:
run_device_diff_regression(device_rollups, target_column='r_sales', baseline_device='desktop')


--- Regression Summary on Differences in 'r_sales' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9647
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.385
Time:                        19:52:08   Log-Likelihood:                -899.82
No. Observations:                  87   AIC:                             1806.
Df Residuals:                      84   BIC:                             1813.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

In [51]:
run_device_diff_regression(device_rollups, target_column='r_sales', baseline_device='tablet')


--- Regression Summary on Differences in 'r_sales' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9647
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.385
Time:                        19:52:09   Log-Likelihood:                -899.82
No. Observations:                  87   AIC:                             1806.
Df Residuals:                      84   BIC:                             1813.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

In [7]:
dev_eff= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\device_efficiency_real.csv", delimiter=';', quotechar='"',
                            engine= 'python')
dev_eff

,Ad_Date,Device,cost_per_click,cost_per_conversion,cost_per_lead,ctr,cost_per_acquisition,cost_per_sale,profit_per_click,leads_per_conversion,lead_conv_rate,roi
0,2024-11-01,desktop,1.588642,32.369630,11.521394,0.029813,32.369630,0.161090,8.273212,2.809524,0.137886,6.2077
1,2024-11-02,desktop,1.480053,30.575032,10.235702,0.028140,30.575032,0.140966,9.019322,2.987097,0.144597,7.0939
2,2024-11-03,desktop,1.511489,31.619749,9.294431,0.029337,31.619749,0.149714,8.584355,3.402010,0.162623,6.6794
3,2024-11-04,desktop,1.526443,31.124537,10.705587,0.032677,31.124537,0.146699,8.878821,2.907317,0.142584,6.8167
4,2024-11-05,desktop,1.267461,29.547688,8.487666,0.035977,29.547688,0.132519,8.296882,3.481250,0.149330,7.5461
...,...,...,...,...,...,...,...,...,...,...,...,...
85,2024-11-26,tablet,1.520913,31.221355,10.707324,0.029814,31.221355,0.134683,9.771598,2.915888,0.142044,7.4248
86,2024-11-27,tablet,1.603347,34.255157,12.463547,0.029334,34.255157,0.160174,8.406662,2.748428,0.128643,6.2432
87,2024-11-28,tablet,1.308027,29.184286,10.355714,0.035037,29.184286,0.138994,8.102625,2.818182,0.126310,7.1945
88,2024-11-29,tablet,1.446378,31.886565,10.849714,0.028156,31.886565,0.140872,8.820935,2.938931,0.133310,7.0986


In [8]:
d_av_eff= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\device_avg_efficiency.csv", delimiter=';', quotechar='"',
                            engine= 'python')
pd.set_option('display.max_columns', None)
d_av_eff

,Device,avg_c/click,avg_c/lead,avg_c/conversions,avg_c/sale,avg_ctr,avg_profit/click,avg_leads/conv.,avg_lead_conv_rate,avg_roi
0,desktop,1.5040,10.4248,31.6909,0.1457,0.0304,8.8195,3.0400,0.1443,6.8641
1,mobile,1.4937,10.4314,31.7848,0.1483,0.0308,8.5817,3.0470,0.1432,6.7453
2,tablet,1.4794,10.2692,32.2640,0.1450,0.0308,8.7228,3.1418,0.1441,6.8962


In [9]:
run_device_regression(dev_eff, target_column='cost_per_click', baseline_device='desktop')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7791
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.462
Time:                        15:05:56   Log-Likelihood:                 86.568
No. Observations:                  90   AIC:                            -167.1
Df Residuals:                      87   BIC:                            -159.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [10]:
run_device_regression(dev_eff, target_column='cost_per_click', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7791
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.462
Time:                        15:06:01   Log-Likelihood:                 86.568
No. Observations:                  90   AIC:                            -167.1
Df Residuals:                      87   BIC:                            -159.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [11]:
run_device_regression(dev_eff, target_column='cost_per_lead', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.5827
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.561
Time:                        15:06:04   Log-Likelihood:                -106.95
No. Observations:                  90   AIC:                             219.9
Df Residuals:                      87   BIC:                             227.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [12]:
run_device_regression(dev_eff, target_column='cost_per_lead', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.5827
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.561
Time:                        15:06:08   Log-Likelihood:                -106.95
No. Observations:                  90   AIC:                             219.9
Df Residuals:                      87   BIC:                             227.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [13]:
run_device_regression(dev_eff, target_column='cost_per_conversion', baseline_device='desktop')


--- Regression Summary for All Keywords ---

                             OLS Regression Results                            
Dep. Variable:     cost_per_conversion   R-squared:                       0.013
Model:                             OLS   Adj. R-squared:                 -0.009
Method:                  Least Squares   F-statistic:                    0.5856
Date:                 Wed, 10 Sep 2025   Prob (F-statistic):              0.559
Time:                         15:07:44   Log-Likelihood:                -213.73
No. Observations:                   90   AIC:                             433.5
Df Residuals:                       87   BIC:                             441.0
Df Model:                            2                                         
Covariance Type:             nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [14]:
run_device_regression(dev_eff, target_column='cost_per_conversion', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                             OLS Regression Results                            
Dep. Variable:     cost_per_conversion   R-squared:                       0.013
Model:                             OLS   Adj. R-squared:                 -0.009
Method:                  Least Squares   F-statistic:                    0.5856
Date:                 Wed, 10 Sep 2025   Prob (F-statistic):              0.559
Time:                         15:07:48   Log-Likelihood:                -213.73
No. Observations:                   90   AIC:                             433.5
Df Residuals:                       87   BIC:                             441.0
Df Model:                            2                                         
Covariance Type:             nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [15]:
run_device_regression(dev_eff, target_column='ctr', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.3431
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.711
Time:                        15:07:51   Log-Likelihood:                 430.22
No. Observations:                  90   AIC:                            -854.4
Df Residuals:                      87   BIC:                            -846.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [16]:
run_device_regression(dev_eff, target_column='ctr', baseline_device='desktop')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.3431
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.711
Time:                        15:07:55   Log-Likelihood:                 430.22
No. Observations:                  90   AIC:                            -854.4
Df Residuals:                      87   BIC:                            -846.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [17]:
run_device_regression(dev_eff, target_column='cost_per_acquisition', baseline_device='desktop')


--- Regression Summary for All Keywords ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.013
Model:                              OLS   Adj. R-squared:                 -0.009
Method:                   Least Squares   F-statistic:                    0.5856
Date:                  Wed, 10 Sep 2025   Prob (F-statistic):              0.559
Time:                          15:08:00   Log-Likelihood:                -213.73
No. Observations:                    90   AIC:                             433.5
Df Residuals:                        87   BIC:                             441.0
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [18]:
run_device_regression(dev_eff, target_column='cost_per_acquisition', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.013
Model:                              OLS   Adj. R-squared:                 -0.009
Method:                   Least Squares   F-statistic:                    0.5856
Date:                  Wed, 10 Sep 2025   Prob (F-statistic):              0.559
Time:                          15:08:02   Log-Likelihood:                -213.73
No. Observations:                    90   AIC:                             433.5
Df Residuals:                        87   BIC:                             441.0
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [19]:
run_device_regression(dev_eff, target_column='cost_per_sale', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.055
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.353
Time:                        15:08:06   Log-Likelihood:                 280.27
No. Observations:                  90   AIC:                            -554.5
Df Residuals:                      87   BIC:                            -547.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [20]:
run_device_regression(dev_eff, target_column='cost_per_sale', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.055
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.353
Time:                        15:08:10   Log-Likelihood:                 280.27
No. Observations:                  90   AIC:                            -554.5
Df Residuals:                      87   BIC:                            -547.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [21]:
run_device_regression(dev_eff, target_column='profit_per_click', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.090
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.341
Time:                        15:08:14   Log-Likelihood:                -104.09
No. Observations:                  90   AIC:                             214.2
Df Residuals:                      87   BIC:                             221.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [22]:
run_device_regression(dev_eff, target_column='profit_per_click', baseline_device='desktop')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.090
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.341
Time:                        15:08:18   Log-Likelihood:                -104.09
No. Observations:                  90   AIC:                             214.2
Df Residuals:                      87   BIC:                             221.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [23]:
run_device_regression(dev_eff, target_column='leads_per_conversion', baseline_device='desktop')


--- Regression Summary for All Keywords ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.032
Model:                              OLS   Adj. R-squared:                  0.010
Method:                   Least Squares   F-statistic:                     1.437
Date:                  Wed, 10 Sep 2025   Prob (F-statistic):              0.243
Time:                          15:08:22   Log-Likelihood:                -23.968
No. Observations:                    90   AIC:                             53.94
Df Residuals:                        87   BIC:                             61.44
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [24]:
run_device_regression(dev_eff, target_column='leads_per_conversion', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.032
Model:                              OLS   Adj. R-squared:                  0.010
Method:                   Least Squares   F-statistic:                     1.437
Date:                  Wed, 10 Sep 2025   Prob (F-statistic):              0.243
Time:                          15:08:26   Log-Likelihood:                -23.968
No. Observations:                    90   AIC:                             53.94
Df Residuals:                        87   BIC:                             61.44
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [25]:
run_device_regression(dev_eff, target_column='lead_conv_rate', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.018
Method:                 Least Squares   F-statistic:                    0.2061
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.814
Time:                        15:08:29   Log-Likelihood:                 272.53
No. Observations:                  90   AIC:                            -539.1
Df Residuals:                      87   BIC:                            -531.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [26]:
run_device_regression(dev_eff, target_column='lead_conv_rate', baseline_device='desktop')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.018
Method:                 Least Squares   F-statistic:                    0.2061
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.814
Time:                        15:08:33   Log-Likelihood:                 272.53
No. Observations:                  90   AIC:                            -539.1
Df Residuals:                      87   BIC:                            -531.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [27]:
run_device_regression(dev_eff, target_column='roi', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.282
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.283
Time:                        15:08:37   Log-Likelihood:                -67.147
No. Observations:                  90   AIC:                             140.3
Df Residuals:                      87   BIC:                             147.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [28]:
run_device_regression(dev_eff, target_column='roi', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.282
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.283
Time:                        15:08:41   Log-Likelihood:                -67.147
No. Observations:                  90   AIC:                             140.3
Df Residuals:                      87   BIC:                             147.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [34]:
run_device_diff_regression(dev_eff, target_column='cost_per_click', baseline_device='mobile')


--- Regression Summary on Differences in 'cost_per_click' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                  0.005072
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.995
Time:                        15:53:06   Log-Likelihood:                 54.558
No. Observations:                  87   AIC:                            -103.1
Df Residuals:                      84   BIC:                            -95.72
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

In [35]:
run_device_diff_regression(dev_eff, target_column='cost_per_click', baseline_device='tablet')


--- Regression Summary on Differences in 'cost_per_click' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                  0.005072
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.995
Time:                        15:53:09   Log-Likelihood:                 54.558
No. Observations:                  87   AIC:                            -103.1
Df Residuals:                      84   BIC:                            -95.72
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

In [36]:
run_device_diff_regression(dev_eff, target_column='cost_per_lead', baseline_device='mobile')


--- Regression Summary on Differences in 'cost_per_lead' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.01322
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.987
Time:                        15:53:13   Log-Likelihood:                -135.61
No. Observations:                  87   AIC:                             277.2
Df Residuals:                      84   BIC:                             284.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [37]:
run_device_diff_regression(dev_eff, target_column='cost_per_lead', baseline_device='tablet')


--- Regression Summary on Differences in 'cost_per_lead' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.01322
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.987
Time:                        15:53:18   Log-Likelihood:                -135.61
No. Observations:                  87   AIC:                             277.2
Df Residuals:                      84   BIC:                             284.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [38]:
run_device_diff_regression(dev_eff, target_column='cost_per_conversion', baseline_device='tablet')


--- Regression Summary on Differences in 'cost_per_conversion' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                  0.003886
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.996
Time:                        15:53:22   Log-Likelihood:                -236.38
No. Observations:                  87   AIC:                             478.8
Df Residuals:                      84   BIC:                             486.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [39]:
run_device_diff_regression(dev_eff, target_column='cost_per_conversion', baseline_device='desktop')


--- Regression Summary on Differences in 'cost_per_conversion' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                  0.003886
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.996
Time:                        15:53:27   Log-Likelihood:                -236.38
No. Observations:                  87   AIC:                             478.8
Df Residuals:                      84   BIC:                             486.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [40]:
run_device_diff_regression(dev_eff, target_column='cost_per_sale', baseline_device='tablet')


--- Regression Summary on Differences in 'cost_per_sale' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02252
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.978
Time:                        15:53:32   Log-Likelihood:                 227.96
No. Observations:                  87   AIC:                            -449.9
Df Residuals:                      84   BIC:                            -442.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [41]:
run_device_diff_regression(dev_eff, target_column='cost_per_sale', baseline_device='mobile')


--- Regression Summary on Differences in 'cost_per_sale' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02252
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.978
Time:                        15:53:37   Log-Likelihood:                 227.96
No. Observations:                  87   AIC:                            -449.9
Df Residuals:                      84   BIC:                            -442.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [42]:
run_device_diff_regression(dev_eff, target_column='ctr', baseline_device='mobile')


--- Regression Summary on Differences in 'ctr' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                   0.06454
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.938
Time:                        15:53:40   Log-Likelihood:                 383.95
No. Observations:                  87   AIC:                            -761.9
Df Residuals:                      84   BIC:                            -754.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [43]:
run_device_diff_regression(dev_eff, target_column='ctr', baseline_device='tablet')


--- Regression Summary on Differences in 'ctr' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                   0.06454
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.938
Time:                        15:54:01   Log-Likelihood:                 383.95
No. Observations:                  87   AIC:                            -761.9
Df Residuals:                      84   BIC:                            -754.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [44]:
run_device_diff_regression(dev_eff, target_column='cost_per_acquisition', baseline_device='desktop')


--- Regression Summary on Differences in 'cost_per_acquisition' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                  0.003886
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.996
Time:                        15:54:05   Log-Likelihood:                -236.38
No. Observations:                  87   AIC:                             478.8
Df Residuals:                      84   BIC:                             486.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

In [45]:
run_device_diff_regression(dev_eff, target_column='cost_per_acquisition', baseline_device='tablet')


--- Regression Summary on Differences in 'cost_per_acquisition' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                  0.003886
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.996
Time:                        15:54:08   Log-Likelihood:                -236.38
No. Observations:                  87   AIC:                             478.8
Df Residuals:                      84   BIC:                             486.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

In [46]:
run_device_diff_regression(dev_eff, target_column='profit_per_click', baseline_device='mobile')


--- Regression Summary on Differences in 'profit_per_click' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                   0.01039
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.990
Time:                        15:54:11   Log-Likelihood:                -139.99
No. Observations:                  87   AIC:                             286.0
Df Residuals:                      84   BIC:                             293.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------

In [47]:
run_device_diff_regression(dev_eff, target_column='profit_per_click', baseline_device='desktop')


--- Regression Summary on Differences in 'profit_per_click' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.024
Method:                 Least Squares   F-statistic:                   0.01039
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.990
Time:                        15:54:14   Log-Likelihood:                -139.99
No. Observations:                  87   AIC:                             286.0
Df Residuals:                      84   BIC:                             293.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------

In [48]:
run_device_diff_regression(dev_eff, target_column='leads_per_conversion', baseline_device='desktop')


--- Regression Summary on Differences in 'leads_per_conversion' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02108
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.979
Time:                        15:54:17   Log-Likelihood:                -50.472
No. Observations:                  87   AIC:                             106.9
Df Residuals:                      84   BIC:                             114.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

In [49]:
run_device_diff_regression(dev_eff, target_column='leads_per_conversion', baseline_device='tablet')


--- Regression Summary on Differences in 'leads_per_conversion' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02108
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.979
Time:                        15:54:20   Log-Likelihood:                -50.472
No. Observations:                  87   AIC:                             106.9
Df Residuals:                      84   BIC:                             114.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

In [50]:
run_device_diff_regression(dev_eff, target_column='lead_conv_rate', baseline_device='desktop')


--- Regression Summary on Differences in 'lead_conv_rate' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.03117
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.969
Time:                        15:54:23   Log-Likelihood:                 243.86
No. Observations:                  87   AIC:                            -481.7
Df Residuals:                      84   BIC:                            -474.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

In [51]:
run_device_diff_regression(dev_eff, target_column='lead_conv_rate', baseline_device='tablet')


--- Regression Summary on Differences in 'lead_conv_rate' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.03117
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.969
Time:                        15:54:26   Log-Likelihood:                 243.86
No. Observations:                  87   AIC:                            -481.7
Df Residuals:                      84   BIC:                            -474.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

In [52]:
run_device_diff_regression(dev_eff, target_column='roi', baseline_device='mobile')


--- Regression Summary on Differences in 'roi' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02129
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.979
Time:                        15:54:29   Log-Likelihood:                -107.81
No. Observations:                  87   AIC:                             221.6
Df Residuals:                      84   BIC:                             229.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [53]:
run_device_diff_regression(dev_eff, target_column='roi', baseline_device='tablet')


--- Regression Summary on Differences in 'roi' by Device ---

                            OLS Regression Results                            
Dep. Variable:            Target_Diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                   0.02129
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.979
Time:                        15:54:33   Log-Likelihood:                -107.81
No. Observations:                  87   AIC:                             221.6
Df Residuals:                      84   BIC:                             229.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [9]:
d_dev_shares= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\daily_device_shares.csv", delimiter=';', quotechar='"',
                            engine= 'python')
d_dev_shares

,Ad_Date,Device,daily_spend,spend_share,daily_clicks,clicks_share,daily_impressions,impressions_share,daily_leads,leads_share,daily_conversions,conversions_share,daily_sales,sales_share
0,2024-11-01,desktop,6117.86,0.4220,3851.0,0.4074,129174.0,0.4395,531.0,0.3945,189.0,0.4375,37978.0,0.4067
1,2024-11-01,mobile,3836.20,0.2646,2510.0,0.2656,69971.0,0.2381,357.0,0.2652,112.0,0.2593,25923.0,0.2776
2,2024-11-01,tablet,4544.41,0.3134,3091.0,0.3270,94776.0,0.3225,458.0,0.3403,131.0,0.3032,29486.0,0.3157
3,2024-11-02,desktop,4739.13,0.3179,3202.0,0.3333,113790.0,0.3360,463.0,0.3171,155.0,0.3138,33619.0,0.3163
4,2024-11-02,mobile,5001.86,0.3355,3229.0,0.3361,118389.0,0.3495,509.0,0.3486,173.0,0.3502,38534.0,0.3625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2024-11-29,mobile,5552.79,0.3640,3680.0,0.3545,116316.0,0.3336,538.0,0.3726,184.0,0.3710,35749.0,0.3399
86,2024-11-29,tablet,4177.14,0.2739,2888.0,0.2782,102572.0,0.2942,385.0,0.2666,131.0,0.2641,29652.0,0.2819
87,2024-11-30,desktop,4133.19,0.3047,2875.0,0.2973,96161.0,0.3078,388.0,0.2885,135.0,0.3245,28328.0,0.2999
88,2024-11-30,mobile,4321.60,0.3186,2943.0,0.3044,100704.0,0.3224,452.0,0.3361,125.0,0.3005,27902.0,0.2954


In [55]:
dev_total_shares= pd.read_csv(r"C:\Users\nehir\Desktop\data shit\ads project data\device_total_shares.csv", delimiter=';', quotechar='"',
                            engine= 'python')
dev_total_shares

,Device,total_spend,all_spend,spend_share,total_clicks,all_clicks,click_share,total_impressions,all_impressions,impression_share,total_leads,all_leads,lead_share,total_conversions,all_conversions,conversion_share,total_sales,all_sales,sales_share
0,desktop,163398.31,482941.32,0.3383,108643.0,323559.0,0.3358,3570321.0,10548269.0,0.3385,15674.0,46538.0,0.3368,5156.0,15138.0,0.3406,1121578.0,3300026.0,0.3399
1,mobile,166901.81,482941.32,0.3456,111737.0,323559.0,0.3453,3624411.0,10548269.0,0.3436,16000.0,46538.0,0.3438,5251.0,15138.0,0.3469,1125797.0,3300026.0,0.3411
2,tablet,152641.20,482941.32,0.3161,103179.0,323559.0,0.3189,3353537.0,10548269.0,0.3179,14864.0,46538.0,0.3194,4731.0,15138.0,0.3125,1052651.0,3300026.0,0.3190


In [ ]:
run_device_regression(dev_eff, target_column='spend_share', baseline_device='desktop')

In [10]:
d_dev_shares_weighted= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\device_total_shares.csv", delimiter=';', quotechar='"',
                            engine= 'python')
d_dev_shares_weighted

,Device,total_spend,all_spend,spend_share,total_clicks,all_clicks,click_share,total_impressions,all_impressions,impression_share,total_leads,all_leads,lead_share,total_conversions,all_conversions,conversion_share,total_sales,all_sales,sales_share
0,desktop,163398.31,482941.32,0.3383,108643.0,323559.0,0.3358,3570321.0,10548269.0,0.3385,15674.0,46538.0,0.3368,5156.0,15138.0,0.3406,1121578.0,3300026.0,0.3399
1,mobile,166901.81,482941.32,0.3456,111737.0,323559.0,0.3453,3624411.0,10548269.0,0.3436,16000.0,46538.0,0.3438,5251.0,15138.0,0.3469,1125797.0,3300026.0,0.3411
2,tablet,152641.20,482941.32,0.3161,103179.0,323559.0,0.3189,3353537.0,10548269.0,0.3179,14864.0,46538.0,0.3194,4731.0,15138.0,0.3125,1052651.0,3300026.0,0.3190


In [68]:
run_device_regression(d_dev_shares, target_column='spend_share', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     2.383
Date:                Wed, 10 Sep 2025   Prob (F-statistic):             0.0983
Time:                        20:33:06   Log-Likelihood:                 127.27
No. Observations:                  90   AIC:                            -248.5
Df Residuals:                      87   BIC:                            -241.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [75]:
run_device_regression(d_dev_shares, target_column='spend_share', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     2.383
Date:                Wed, 10 Sep 2025   Prob (F-statistic):             0.0983
Time:                        21:14:33   Log-Likelihood:                 127.27
No. Observations:                  90   AIC:                            -248.5
Df Residuals:                      87   BIC:                            -241.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [76]:
run_device_regression(d_dev_shares, target_column='clicks_share', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     1.945
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.149
Time:                        21:16:44   Log-Likelihood:                 129.69
No. Observations:                  90   AIC:                            -253.4
Df Residuals:                      87   BIC:                            -245.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [77]:
run_device_regression(d_dev_shares, target_column='clicks_share', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     1.945
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.149
Time:                        21:16:57   Log-Likelihood:                 129.69
No. Observations:                  90   AIC:                            -253.4
Df Residuals:                      87   BIC:                            -245.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [78]:
run_device_regression(d_dev_shares, target_column='impressions_share', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     1.895
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.156
Time:                        21:20:47   Log-Likelihood:                 126.19
No. Observations:                  90   AIC:                            -246.4
Df Residuals:                      87   BIC:                            -238.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [79]:
run_device_regression(d_dev_shares, target_column='impressions_share', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     1.895
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.156
Time:                        21:20:48   Log-Likelihood:                 126.19
No. Observations:                  90   AIC:                            -246.4
Df Residuals:                      87   BIC:                            -238.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [80]:
run_device_regression(d_dev_shares, target_column='leads_share', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.831
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.166
Time:                        21:20:53   Log-Likelihood:                 130.31
No. Observations:                  90   AIC:                            -254.6
Df Residuals:                      87   BIC:                            -247.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [81]:
run_device_regression(d_dev_shares, target_column='leads_share', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.831
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.166
Time:                        21:20:56   Log-Likelihood:                 130.31
No. Observations:                  90   AIC:                            -254.6
Df Residuals:                      87   BIC:                            -247.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [82]:
run_device_regression(d_dev_shares, target_column='conversions_share', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     3.282
Date:                Wed, 10 Sep 2025   Prob (F-statistic):             0.0423
Time:                        21:20:57   Log-Likelihood:                 126.18
No. Observations:                  90   AIC:                            -246.4
Df Residuals:                      87   BIC:                            -238.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [83]:
run_device_regression(d_dev_shares, target_column='conversions_share', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     3.282
Date:                Wed, 10 Sep 2025   Prob (F-statistic):             0.0423
Time:                        21:21:20   Log-Likelihood:                 126.18
No. Observations:                  90   AIC:                            -246.4
Df Residuals:                      87   BIC:                            -238.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [84]:
run_device_regression(d_dev_shares, target_column='sales_share', baseline_device='mobile')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.822
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.168
Time:                        21:22:39   Log-Likelihood:                 132.10
No. Observations:                  90   AIC:                            -258.2
Df Residuals:                      87   BIC:                            -250.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [85]:
run_device_regression(d_dev_shares, target_column='sales_share', baseline_device='tablet')


--- Regression Summary for All Keywords ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.822
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.168
Time:                        21:22:41   Log-Likelihood:                 132.10
No. Observations:                  90   AIC:                            -258.2
Df Residuals:                      87   BIC:                            -250.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [83]:
def run_keyword_regression(dataframe, target_column='ConversionRate', baseline_keyword=None):

    subset = dataframe

    if baseline_keyword is not None and baseline_keyword in subset['Keyword'].unique():
        subset['Keyword'] = pd.Categorical(
            subset['Keyword'],
            categories=[baseline_keyword] + [k for k in subset['Keyword'].unique() if k != baseline_keyword],
            ordered=True
        )

    # One-hot encode keywords, dropping baseline
    X = pd.get_dummies(subset['Keyword'], drop_first=True)
    X = sm.add_constant(X).astype(float)

    # Target variable
    y = subset[target_column].astype(float)

    # Run regression
    model = sm.OLS(y, X).fit()

    print(f"\n--- Regression Summary for Keywords ---\n")
    print(model.summary())

    return model

In [11]:
keyw_rollups= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\keyw_rollups.csv", delimiter=';', quotechar='"',
                            engine= 'python')
keyw_rollups

,Ad_Date,Keyword,daily_spend,rolling_cost,daily_clicks,r_clicks,daily_impressions,r_impressions,daily_leads,r_leads,daily_conversions,r_conversions,daily_sales,r_sales,avg_conversions
0,2024-11-01,analytics for data,1742.75,1742.75,1083.0,1083.0,31375.0,31375.0,160.0,160.0,60.0,60.0,11384.0,11384.0,0.055750
1,2024-11-02,analytics for data,3300.14,5042.89,1762.0,2845.0,63464.0,94839.0,266.0,426.0,98.0,158.0,18340.0,29724.0,0.056000
2,2024-11-03,analytics for data,2407.00,7449.89,1804.0,4649.0,54951.0,149790.0,256.0,682.0,85.0,243.0,17803.0,47527.0,0.051250
3,2024-11-04,analytics for data,2843.71,10293.60,2077.0,6726.0,65320.0,215110.0,264.0,946.0,76.0,319.0,19766.0,67293.0,0.040231
4,2024-11-05,analytics for data,2982.19,13275.79,2206.0,8932.0,66267.0,281377.0,339.0,1285.0,91.0,410.0,21983.0,89276.0,0.041818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2024-11-26,online data analytics,3732.56,73905.41,2431.0,49201.0,85765.0,1580100.0,372.0,7069.0,128.0,2364.0,23523.0,488207.0,0.055133
176,2024-11-27,online data analytics,2184.03,76089.44,1195.0,50396.0,42103.0,1622203.0,180.0,7249.0,69.0,2433.0,12512.0,500719.0,0.067222
177,2024-11-28,online data analytics,2145.68,78235.12,1665.0,52061.0,49431.0,1671634.0,218.0,7467.0,70.0,2503.0,15684.0,516403.0,0.042000
178,2024-11-29,online data analytics,3460.89,81696.01,2438.0,54499.0,83613.0,1755247.0,294.0,7761.0,117.0,2620.0,24569.0,540972.0,0.048250


In [12]:
keyw_r_avgs= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\keyw_rollups_avgs.csv", delimiter=';', quotechar='"',
                            engine= 'python')
keyw_r_avgs

,Keyword,avg_spendings,avg_clicks,avg_impressions,avg_leads,avg_conversions,avg_sales,avg_daily_conv_rate
0,analytics for data,2683.01,1791.90,59558.50,261.70,82.13,18447.87,0.05
1,data analytics course,2690.50,1834.07,59033.87,262.47,85.07,18733.00,0.05
2,data analytics online,2513.53,1721.37,55709.10,249.97,77.57,17072.63,0.05
3,data analytics training,2511.68,1693.43,55855.67,236.60,80.97,17344.53,0.05
4,learn data analytics,2862.29,1845.47,60360.13,270.43,88.37,19545.03,0.05
5,online data analytics,2837.04,1899.07,61091.70,270.10,90.50,18857.80,0.05


In [111]:
run_keyword_regression(keyw_rollups, target_column='daily_spend', baseline_keyword='learn data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.249
Date:                Wed, 10 Sep 2025   Prob (F-statistic):              0.288
Time:                        21:55:55   Log-Likelihood:                -1441.6
No. Observations:                 180   AIC:                             2895.
Df Residuals:                     174   BIC:                             2914.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [7]:
run_keyword_regression(keyw_rollups, target_column='daily_spend', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.249
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.288
Time:                        01:21:23   Log-Likelihood:                -1441.6
No. Observations:                 180   AIC:                             2895.
Df Residuals:                     174   BIC:                             2914.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [8]:
run_keyword_regression(keyw_rollups, target_column='daily_clicks', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7595
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.580
Time:                        01:21:29   Log-Likelihood:                -1368.0
No. Observations:                 180   AIC:                             2748.
Df Residuals:                     174   BIC:                             2767.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [9]:
run_keyword_regression(keyw_rollups, target_column='daily_clicks', baseline_keyword='online data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7595
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.580
Time:                        01:21:33   Log-Likelihood:                -1368.0
No. Observations:                 180   AIC:                             2748.
Df Residuals:                     174   BIC:                             2767.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [10]:
run_keyword_regression(keyw_rollups, target_column='daily_impressions', baseline_keyword='data analytics online')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.5973
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.702
Time:                        01:21:36   Log-Likelihood:                -1997.7
No. Observations:                 180   AIC:                             4007.
Df Residuals:                     174   BIC:                             4026.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [11]:
run_keyword_regression(keyw_rollups, target_column='daily_impressions', baseline_keyword='online data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.5973
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.702
Time:                        01:21:40   Log-Likelihood:                -1997.7
No. Observations:                 180   AIC:                             4007.
Df Residuals:                     174   BIC:                             4026.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [12]:
run_keyword_regression(keyw_rollups, target_column='daily_leads', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9868
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.427
Time:                        01:21:41   Log-Likelihood:                -1022.4
No. Observations:                 180   AIC:                             2057.
Df Residuals:                     174   BIC:                             2076.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [13]:
run_keyword_regression(keyw_rollups, target_column='daily_leads', baseline_keyword='learn data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9868
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.427
Time:                        01:21:44   Log-Likelihood:                -1022.4
No. Observations:                 180   AIC:                             2057.
Df Residuals:                     174   BIC:                             2076.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [14]:
run_keyword_regression(keyw_rollups, target_column='daily_conversions', baseline_keyword='data analytics online')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.220
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.301
Time:                        01:21:48   Log-Likelihood:                -823.87
No. Observations:                 180   AIC:                             1660.
Df Residuals:                     174   BIC:                             1679.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [15]:
run_keyword_regression(keyw_rollups, target_column='daily_conversions', baseline_keyword='online data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.220
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.301
Time:                        01:24:15   Log-Likelihood:                -823.87
No. Observations:                 180   AIC:                             1660.
Df Residuals:                     174   BIC:                             1679.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [16]:
run_keyword_regression(keyw_rollups, target_column='daily_sales', baseline_keyword='learn data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.011
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.413
Time:                        01:24:46   Log-Likelihood:                -1791.0
No. Observations:                 180   AIC:                             3594.
Df Residuals:                     174   BIC:                             3613.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [17]:
run_keyword_regression(keyw_rollups, target_column='daily_sales', baseline_keyword='data analytics online')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.011
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.413
Time:                        01:24:50   Log-Likelihood:                -1791.0
No. Observations:                 180   AIC:                             3594.
Df Residuals:                     174   BIC:                             3613.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [13]:
keyw_eff= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\keyw_efficiency.csv", delimiter=';', quotechar='"',
                            engine= 'python')
keyw_eff

,Ad_Date,Keyword,cost_per_click,cost_per_lead,ctr,cost_per_acquisition,cost_per_sale,profit_per_click,leads_per_conversion,lead_conv_rate,roi
0,2024-11-01,analytics for data,1.6092,10.8922,0.0345,29.0458,0.1531,8.9024,2.6667,0.1477,6.5322
1,2024-11-02,analytics for data,1.8730,12.4065,0.0278,33.6749,0.1799,8.5357,2.7143,0.1510,5.5573
2,2024-11-03,analytics for data,1.3343,9.4023,0.0328,28.3176,0.1352,8.5344,3.0118,0.1419,7.3963
3,2024-11-04,analytics for data,1.3691,10.7716,0.0318,37.4172,0.1439,8.1475,3.4737,0.1271,6.9508
4,2024-11-05,analytics for data,1.3519,8.7970,0.0333,32.7713,0.1357,8.6132,3.7253,0.1537,7.3714
...,...,...,...,...,...,...,...,...,...,...,...
175,2024-11-26,online data analytics,1.5354,10.0338,0.0283,29.1606,0.1587,8.1409,2.9062,0.1530,6.3021
176,2024-11-27,online data analytics,1.8276,12.1335,0.0284,31.6526,0.1746,8.6427,2.6087,0.1506,5.7289
177,2024-11-28,online data analytics,1.2887,9.8426,0.0337,30.6526,0.1368,8.1311,3.1143,0.1309,7.3096
178,2024-11-29,online data analytics,1.4196,11.7717,0.0292,29.5803,0.1409,8.6580,2.5128,0.1206,7.0990


In [19]:
mean_df = keyw_eff.groupby('Keyword').mean(numeric_only=True).reset_index()
mean_df

,Keyword,cost_per_click,cost_per_lead,ctr,cost_per_acquisition,cost_per_sale,profit_per_click,leads_per_conversion,lead_conv_rate,roi
0,analytics for data,1.504163,10.336110,0.030303,33.490297,0.148023,8.788953,3.263167,0.146267,6.880960
1,data analytics course,1.476610,10.337703,0.031180,31.808983,0.145053,8.772183,3.094923,0.143707,6.952147
2,data analytics online,1.460983,10.177230,0.031113,32.841670,0.148210,8.499183,3.256913,0.144837,6.836517
3,data analytics training,1.483980,10.700893,0.030490,31.187423,0.144757,8.818403,2.940647,0.140210,6.968557
4,learn data analytics,1.559913,10.610217,0.030623,32.832477,0.147073,9.138927,3.113100,0.147947,6.885173
5,online data analytics,1.511967,10.506987,0.031053,31.377533,0.151583,8.512510,2.995910,0.144380,6.656743


In [20]:
run_keyword_regression(keyw_eff, target_column='cost_per_click', baseline_keyword='data analytics online')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.649
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.150
Time:                        01:37:09   Log-Likelihood:                 90.709
No. Observations:                 180   AIC:                            -169.4
Df Residuals:                     174   BIC:                            -150.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [21]:
run_keyword_regression(keyw_eff, target_column='cost_per_click', baseline_keyword='learn data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.649
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.150
Time:                        01:38:11   Log-Likelihood:                 90.709
No. Observations:                 180   AIC:                            -169.4
Df Residuals:                     174   BIC:                            -150.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [22]:
run_keyword_regression(keyw_eff, target_column='cost_per_lead', baseline_keyword='analytics for data')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9591
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.444
Time:                        01:39:24   Log-Likelihood:                -268.71
No. Observations:                 180   AIC:                             549.4
Df Residuals:                     174   BIC:                             568.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [23]:
run_keyword_regression(keyw_eff, target_column='cost_per_lead', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9591
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.444
Time:                        01:39:27   Log-Likelihood:                -268.71
No. Observations:                 180   AIC:                             549.4
Df Residuals:                     174   BIC:                             568.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [24]:
run_keyword_regression(keyw_eff, target_column='ctr', baseline_keyword='data analytics course')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.5054
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.772
Time:                        01:39:29   Log-Likelihood:                 803.01
No. Observations:                 180   AIC:                            -1594.
Df Residuals:                     174   BIC:                            -1575.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [25]:
run_keyword_regression(keyw_eff, target_column='ctr', baseline_keyword='analytics for data')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.5054
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.772
Time:                        01:39:31   Log-Likelihood:                 803.01
No. Observations:                 180   AIC:                            -1594.
Df Residuals:                     174   BIC:                            -1575.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [26]:
run_keyword_regression(keyw_eff, target_column='cost_per_acquisition', baseline_keyword='analytics for data')


--- Regression Summary for Keywords ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.042
Model:                              OLS   Adj. R-squared:                  0.015
Method:                   Least Squares   F-statistic:                     1.536
Date:                  Fri, 12 Sep 2025   Prob (F-statistic):              0.181
Time:                          01:39:33   Log-Likelihood:                -506.49
No. Observations:                   180   AIC:                             1025.
Df Residuals:                       174   BIC:                             1044.
Df Model:                             5                                         
Covariance Type:              nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

In [27]:
run_keyword_regression(keyw_eff, target_column='cost_per_acquisition', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.042
Model:                              OLS   Adj. R-squared:                  0.015
Method:                   Least Squares   F-statistic:                     1.536
Date:                  Fri, 12 Sep 2025   Prob (F-statistic):              0.181
Time:                          01:40:28   Log-Likelihood:                -506.49
No. Observations:                   180   AIC:                             1025.
Df Residuals:                       174   BIC:                             1044.
Df Model:                             5                                         
Covariance Type:              nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [28]:
run_keyword_regression(keyw_eff, target_column='cost_per_sale', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.6790
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.640
Time:                        01:40:54   Log-Likelihood:                 485.50
No. Observations:                 180   AIC:                            -959.0
Df Residuals:                     174   BIC:                            -939.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [29]:
run_keyword_regression(keyw_eff, target_column='cost_per_sale', baseline_keyword='online data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.6790
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.640
Time:                        01:40:56   Log-Likelihood:                 485.50
No. Observations:                 180   AIC:                            -959.0
Df Residuals:                     174   BIC:                            -939.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [30]:
run_keyword_regression(keyw_eff, target_column='profit_per_click', baseline_keyword='learn data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1.283
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.274
Time:                        01:40:57   Log-Likelihood:                -275.72
No. Observations:                 180   AIC:                             563.4
Df Residuals:                     174   BIC:                             582.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [31]:
run_keyword_regression(keyw_eff, target_column='profit_per_click', baseline_keyword='data analytics online')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1.283
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.274
Time:                        01:41:46   Log-Likelihood:                -275.72
No. Observations:                 180   AIC:                             563.4
Df Residuals:                     174   BIC:                             582.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [32]:
run_keyword_regression(keyw_eff, target_column='leads_per_conversion', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.075
Model:                              OLS   Adj. R-squared:                  0.049
Method:                   Least Squares   F-statistic:                     2.835
Date:                  Fri, 12 Sep 2025   Prob (F-statistic):             0.0173
Time:                          01:42:24   Log-Likelihood:                -100.02
No. Observations:                   180   AIC:                             212.0
Df Residuals:                       174   BIC:                             231.2
Df Model:                             5                                         
Covariance Type:              nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [33]:
run_keyword_regression(keyw_eff, target_column='leads_per_conversion', baseline_keyword='analytics for data')


--- Regression Summary for Keywords ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.075
Model:                              OLS   Adj. R-squared:                  0.049
Method:                   Least Squares   F-statistic:                     2.835
Date:                  Fri, 12 Sep 2025   Prob (F-statistic):             0.0173
Time:                          01:43:32   Log-Likelihood:                -100.02
No. Observations:                   180   AIC:                             212.0
Df Residuals:                       174   BIC:                             231.2
Df Model:                             5                                         
Covariance Type:              nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

In [34]:
run_keyword_regression(keyw_eff, target_column='lead_conv_rate', baseline_keyword='learn data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.7753
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.569
Time:                        01:43:57   Log-Likelihood:                 489.12
No. Observations:                 180   AIC:                            -966.2
Df Residuals:                     174   BIC:                            -947.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [35]:
run_keyword_regression(keyw_eff, target_column='lead_conv_rate', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.7753
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.569
Time:                        01:43:59   Log-Likelihood:                 489.12
No. Observations:                 180   AIC:                            -966.2
Df Residuals:                     174   BIC:                            -947.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [36]:
run_keyword_regression(keyw_eff, target_column='roi', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.6852
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.635
Time:                        01:44:01   Log-Likelihood:                -199.03
No. Observations:                 180   AIC:                             410.1
Df Residuals:                     174   BIC:                             429.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [37]:
run_keyword_regression(keyw_eff, target_column='roi', baseline_keyword='online data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.6852
Date:                Fri, 12 Sep 2025   Prob (F-statistic):              0.635
Time:                        01:44:03   Log-Likelihood:                -199.03
No. Observations:                 180   AIC:                             410.1
Df Residuals:                     174   BIC:                             429.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [14]:
keyw_shares= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\keyw_shares.csv", delimiter=';', quotechar='"',
                            engine= 'python')
keyw_shares

,Ad_Date,Keyword,daily_spend,spend_share,daily_clicks,clicks_share,daily_impressions,impressions_share,daily_leads,leads_share,daily_conversions,conversions_share,daily_sales,sales_share
0,2024-11-01,analytics for data,1742.75,0.1202,1083.0,0.1146,31375.0,0.1067,160.0,0.1189,60.0,0.1389,11384.0,0.1219
1,2024-11-02,analytics for data,3300.14,0.2214,1762.0,0.1834,63464.0,0.1874,266.0,0.1822,98.0,0.1984,18340.0,0.1725
2,2024-11-03,analytics for data,2407.00,0.1610,1804.0,0.1767,54951.0,0.1612,256.0,0.1714,85.0,0.1881,17803.0,0.1744
3,2024-11-04,analytics for data,2843.71,0.1747,2077.0,0.1820,65320.0,0.1900,264.0,0.1654,76.0,0.1573,19766.0,0.1749
4,2024-11-05,analytics for data,2982.19,0.1739,2206.0,0.1817,66267.0,0.1742,339.0,0.1994,91.0,0.1727,21983.0,0.1788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2024-11-26,online data analytics,3732.56,0.2114,2431.0,0.2044,85765.0,0.2200,372.0,0.2139,128.0,0.2203,23523.0,0.1974
176,2024-11-27,online data analytics,2184.03,0.1372,1195.0,0.1217,42103.0,0.1258,180.0,0.1313,69.0,0.1366,12512.0,0.1193
177,2024-11-28,online data analytics,2145.68,0.1599,1665.0,0.1745,49431.0,0.1686,218.0,0.1722,70.0,0.1628,15684.0,0.1631
178,2024-11-29,online data analytics,3460.89,0.2269,2438.0,0.2349,83613.0,0.2398,294.0,0.2036,117.0,0.2359,24569.0,0.2336


In [65]:
mean_df = keyw_shares.groupby('Keyword').mean(numeric_only=True).reset_index()
mean_df

,Keyword,daily_spend,spend_share,daily_clicks,clicks_share,daily_impressions,impressions_share,daily_leads,leads_share,daily_conversions,conversions_share,daily_sales,sales_share
0,analytics for data,2683.014000,0.166180,1791.900000,0.165980,59558.500000,0.168510,261.700000,0.167970,82.133333,0.162953,18447.866667,0.166633
1,data analytics course,2690.498000,0.167313,1834.066667,0.169943,59033.866667,0.167947,262.466667,0.169347,85.066667,0.169090,18733.000000,0.170170
2,data analytics online,2513.527000,0.156827,1721.366667,0.160280,55709.100000,0.158763,249.966667,0.161133,77.566667,0.154050,17072.633333,0.155073
3,data analytics training,2511.675667,0.157197,1693.433333,0.158213,55855.666667,0.160163,236.600000,0.153247,80.966667,0.161230,17344.533333,0.159113
4,learn data analytics,2862.289667,0.176877,1845.466667,0.170497,60360.133333,0.171237,270.433333,0.174167,88.366667,0.173857,19545.033333,0.177590
5,online data analytics,2837.039667,0.175597,1899.066667,0.175090,61091.700000,0.173387,270.100000,0.174147,90.500000,0.178820,18857.800000,0.171433


In [8]:
run_keyword_regression(keyw_shares, target_column='spend_share', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.352
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.245
Time:                        18:57:07   Log-Likelihood:                 324.42
No. Observations:                 180   AIC:                            -636.8
Df Residuals:                     174   BIC:                            -617.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [9]:
run_keyword_regression(keyw_shares, target_column='spend_share', baseline_keyword='learn data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.352
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.245
Time:                        18:57:22   Log-Likelihood:                 324.42
No. Observations:                 180   AIC:                            -636.8
Df Residuals:                     174   BIC:                            -617.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [10]:
run_keyword_regression(keyw_shares, target_column='clicks_share', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7507
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.587
Time:                        18:57:26   Log-Likelihood:                 323.13
No. Observations:                 180   AIC:                            -634.3
Df Residuals:                     174   BIC:                            -615.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [11]:
run_keyword_regression(keyw_shares, target_column='clicks_share', baseline_keyword='online data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7507
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.587
Time:                        18:57:28   Log-Likelihood:                 323.13
No. Observations:                 180   AIC:                            -634.3
Df Residuals:                     174   BIC:                            -615.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [12]:
run_keyword_regression(keyw_shares, target_column='impressions_share', baseline_keyword='data analytics online')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.6368
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.672
Time:                        18:57:29   Log-Likelihood:                 323.90
No. Observations:                 180   AIC:                            -635.8
Df Residuals:                     174   BIC:                            -616.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [13]:
run_keyword_regression(keyw_shares, target_column='impressions_share', baseline_keyword='online data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.6368
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.672
Time:                        18:57:31   Log-Likelihood:                 323.90
No. Observations:                 180   AIC:                            -635.8
Df Residuals:                     174   BIC:                            -616.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [14]:
run_keyword_regression(keyw_shares, target_column='leads_share', baseline_keyword='data analytics training')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.234
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.295
Time:                        18:57:33   Log-Likelihood:                 326.27
No. Observations:                 180   AIC:                            -640.5
Df Residuals:                     174   BIC:                            -621.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

In [15]:
run_keyword_regression(keyw_shares, target_column='leads_share', baseline_keyword='learn data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.234
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.295
Time:                        18:58:13   Log-Likelihood:                 326.27
No. Observations:                 180   AIC:                            -640.5
Df Residuals:                     174   BIC:                            -621.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [16]:
run_keyword_regression(keyw_shares, target_column='conversions_share', baseline_keyword='data analytics online')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.373
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.237
Time:                        18:58:53   Log-Likelihood:                 317.34
No. Observations:                 180   AIC:                            -622.7
Df Residuals:                     174   BIC:                            -603.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [17]:
run_keyword_regression(keyw_shares, target_column='conversions_share', baseline_keyword='online data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.373
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.237
Time:                        18:59:45   Log-Likelihood:                 317.34
No. Observations:                 180   AIC:                            -622.7
Df Residuals:                     174   BIC:                            -603.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [18]:
run_keyword_regression(keyw_shares, target_column='sales_share', baseline_keyword='learn data analytics')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.179
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.321
Time:                        19:00:04   Log-Likelihood:                 318.47
No. Observations:                 180   AIC:                            -624.9
Df Residuals:                     174   BIC:                            -605.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [19]:
run_keyword_regression(keyw_shares, target_column='sales_share', baseline_keyword='data analytics online')


--- Regression Summary for Keywords ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.179
Date:                Sat, 13 Sep 2025   Prob (F-statistic):              0.321
Time:                        19:00:33   Log-Likelihood:                 318.47
No. Observations:                 180   AIC:                            -624.9
Df Residuals:                     174   BIC:                            -605.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [15]:
keyw_dev_rollups= pd.read_csv(r"C:\Users\nehir\Desktop\analyst folder\ads project data\keyw_device_rollups.csv", delimiter=';', quotechar='"',
                            engine= 'python')
keyw_dev_rollups

,Ad_Date,Keyword,Device,daily_spend,rolling_cost,daily_clicks,r_clicks,daily_impressions,r_impressions,daily_leads,r_leads,daily_conversions,r_conversions,daily_sales,r_sales,avg_conversions
0,2024-11-01,analytics for data,desktop,1272.02,1272.02,820.0,820.0,24359.0,24359.0,119.0,119.0,42.0,42.0,8698.0,8698.0,0.056167
1,2024-11-02,analytics for data,desktop,1539.67,2811.69,861.0,1681.0,31313.0,55672.0,130.0,249.0,42.0,84.0,8352.0,17050.0,0.051571
2,2024-11-03,analytics for data,desktop,841.98,3653.67,515.0,2196.0,21069.0,76741.0,88.0,337.0,26.0,110.0,5477.0,22527.0,0.053500
3,2024-11-04,analytics for data,desktop,1133.05,4786.72,691.0,2887.0,21814.0,98555.0,89.0,426.0,31.0,141.0,7417.0,29944.0,0.053500
4,2024-11-05,analytics for data,desktop,889.22,5675.94,813.0,3700.0,19470.0,118025.0,121.0,547.0,30.0,171.0,7583.0,37527.0,0.040667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,2024-11-26,online data analytics,tablet,1081.37,25256.17,702.0,17037.0,26089.0,536536.0,106.0,2345.0,38.0,784.0,9388.0,173957.0,0.059000
528,2024-11-27,online data analytics,tablet,474.20,25730.37,279.0,17316.0,8766.0,545302.0,28.0,2373.0,20.0,804.0,2834.0,176791.0,0.087000
529,2024-11-28,online data analytics,tablet,412.62,26142.99,410.0,17726.0,14065.0,559367.0,43.0,2416.0,21.0,825.0,3706.0,180497.0,0.042000
530,2024-11-29,online data analytics,tablet,1059.22,27202.21,834.0,18560.0,31006.0,590373.0,93.0,2509.0,41.0,866.0,9413.0,189910.0,0.050667


In [24]:
mean_df = keyw_dev_rollups.groupby(['Keyword', 'Device']).mean(numeric_only=True).reset_index()
mean_df

,Keyword,Device,daily_spend,rolling_cost,daily_clicks,r_clicks,daily_impressions,r_impressions,daily_leads,r_leads,daily_conversions,r_conversions,daily_sales,r_sales,avg_conversions
0,analytics for data,desktop,956.325172,15784.920345,617.241379,10092.896552,20974.172414,337008.379310,87.206897,1422.724138,29.965517,491.034483,6623.344828,107896.310345,0.050695
1,analytics for data,mobile,980.791034,14860.337586,662.965517,9876.517241,21612.827586,321537.689655,97.137931,1435.931034,28.724138,430.137931,6640.862069,98489.620690,0.043671
2,analytics for data,tablet,810.468333,12086.428000,554.366667,8224.333333,18391.066667,267700.733333,83.500000,1258.266667,25.400000,372.100000,5625.800000,81609.533333,0.048154
3,data analytics course,desktop,950.776333,14849.506667,666.633333,10346.933333,20865.066667,324818.300000,90.966667,1454.266667,31.133333,477.200000,6871.966667,104965.066667,0.047843
4,data analytics course,mobile,783.619667,12794.040333,525.000000,8580.233333,17412.500000,291114.000000,77.066667,1271.866667,23.733333,395.066667,5484.366667,91288.100000,0.050119
5,data analytics course,tablet,956.102000,13075.190333,642.433333,8764.966667,20756.300000,283005.066667,94.433333,1281.333333,30.200000,411.833333,6376.666667,89845.366667,0.050876
6,data analytics online,desktop,898.817333,14227.683667,615.333333,9670.366667,19332.100000,306952.633333,90.800000,1445.733333,27.266667,434.333333,6081.433333,98575.033333,0.044834
7,data analytics online,mobile,806.789333,12698.600000,553.333333,8697.266667,18418.000000,279943.466667,75.933333,1161.866667,25.633333,389.800000,5193.866667,80662.200000,0.045197
8,data analytics online,tablet,835.779655,12313.192069,571.758621,8379.862069,18578.275862,275479.793103,86.103448,1298.344828,25.517241,383.793103,5997.241379,88939.896552,0.047278
9,data analytics training,desktop,952.045862,14372.787241,643.517241,9776.275862,21723.517241,326430.000000,92.172414,1373.413793,32.103448,483.379310,6595.068966,100141.241379,0.049241


In [27]:
# List of numeric columns only
numeric_cols = mean_df.select_dtypes(include='number').columns

# Store results
extremes = []

for col in numeric_cols:
    max_val = mean_df[col].max()
    min_val = mean_df[col].min()
    
    # Get the row where max and min occur
    max_row = mean_df.loc[mean_df[col] == max_val].iloc[0]
    min_row = mean_df.loc[mean_df[col] == min_val].iloc[0]
    
    # Concatenate 'Keyword' and 'Device' for identification
    max_id = f"{max_row['Keyword']}_{max_row['Device']}"
    min_id = f"{min_row['Keyword']}_{min_row['Device']}"
    
    extremes.append({
        'column': col,
        'max_at': max_id,
        'min_at': min_id
    })

# Create result DataFrame
extremes_df = pd.DataFrame(extremes)

# Display
print(extremes_df)

               column                       max_at  \
0         daily_spend  learn data analytics_mobile   
1        rolling_cost  learn data analytics_mobile   
2        daily_clicks  learn data analytics_mobile   
3            r_clicks  learn data analytics_mobile   
4   daily_impressions  learn data analytics_mobile   
5       r_impressions  learn data analytics_mobile   
6         daily_leads  learn data analytics_mobile   
7             r_leads  learn data analytics_mobile   
8   daily_conversions  learn data analytics_mobile   
9       r_conversions  learn data analytics_mobile   
10        daily_sales  learn data analytics_mobile   
11            r_sales  learn data analytics_mobile   
12    avg_conversions  learn data analytics_mobile   

                            min_at  
0   data analytics training_tablet  
1   data analytics training_tablet  
2   data analytics training_tablet  
3   data analytics training_tablet  
4   data analytics training_tablet  
5   data analytics tr

In [102]:
def run_keyword_device_regression(dataframe, keyword_col='Keyword', device_col='Device', target_col='', baseline=None):
    # Make a copy of the input dataframe to avoid modifying the original
    df = dataframe.copy()
    
    # Create a combined column for Keyword and Device (e.g., "data analytics_mobile")
    df['Keyword_Device'] = df[keyword_col].astype(str) + '_' + df[device_col].astype(str)
    
    # Convert the categorical Keyword_Device column into dummy/indicator variables (one-hot encoding)
    dummies = pd.get_dummies(df['Keyword_Device']).astype(float)
    
    # Optionally drop a baseline category to avoid multicollinearity
    if baseline is not None:
        dummies = dummies.drop(columns=baseline)
    
    # Add an intercept/constant term to the regression design matrix
    X = sm.add_constant(dummies)
    
    # Define the target variable (dependent variable), ensuring it’s numeric
    y = df[target_col].astype(float)
    
    # Fit an Ordinary Least Squares (OLS) regression model
    model = sm.OLS(y, X).fit()
    
    # Extract the regression results summary table (coefficients, std errors, p-values, etc.)
    sig_results = model.summary2().tables[1]
    
    # Filter results to keep only statistically significant predictors (p-value ≤ 0.002)
    filtered = sig_results[sig_results['P>|t|'] <= 0.002]
    
    # Print the significant results
    print(filtered)
    
    # Return the fitted regression model object for further inspection or use
    return model


In [86]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_spend', 
                                      baseline='learn data analytics_mobile')

                                      Coef.    Std.Err.          t  \
const                           1140.675172   80.670356  14.139955   
data analytics course_mobile    -357.055506  113.130407  -3.156141   
data analytics training_tablet  -435.585517  114.085111  -3.818075   

                                       P>|t|      [0.025       0.975]  
const                           1.403512e-38  982.190998  1299.159347  
data analytics course_mobile    1.692462e-03 -579.310373  -134.800639  
data analytics training_tablet  1.509068e-04 -659.715986  -211.455048  


In [88]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_spend', 
                                      baseline='data analytics training_tablet')

                                  Coef.    Std.Err.         t         P>|t|  \
const                        705.089655   80.670356  8.740381  3.302119e-17   
learn data analytics_mobile  435.585517  114.085111  3.818075  1.509068e-04   

                                 [0.025      0.975]  
const                        546.605481  863.573830  
learn data analytics_mobile  211.455048  659.715986  


In [87]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_clicks', 
                                      baseline='data analytics training_tablet')

                                  Coef.   Std.Err.         t         P>|t|  \
const                        477.275862  54.629766  8.736553  3.400959e-17   
learn data analytics_mobile  255.137931  77.258155  3.302408  1.025287e-03   

                                 [0.025      0.975]  
const                        369.950771  584.600953  
learn data analytics_mobile  103.357331  406.918531  


In [89]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_clicks', 
                                      baseline='learn data analytics_mobile')

                                     Coef.   Std.Err.          t  \
const                           732.413793  54.629766  13.406863   
data analytics training_tablet -255.137931  77.258155  -3.302408   

                                       P>|t|      [0.025      0.975]  
const                           2.325227e-35  625.088702  839.738884  
data analytics training_tablet  1.025287e-03 -406.918531 -103.357331  


In [90]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_impressions', 
                                      baseline='data analytics training_tablet')

                                    Coef.     Std.Err.         t  \
const                        15676.275862  1752.322735  8.945998   
learn data analytics_mobile   8143.310345  2478.158577  3.286033   

                                    P>|t|        [0.025        0.975]  
const                        6.681563e-18  12233.680139  19118.871585  
learn data analytics_mobile  1.085473e-03   3274.744784  13011.875906  


In [91]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_impressions', 
                                      baseline='learn data analytics_mobile')

                                       Coef.     Std.Err.          t  \
const                           23819.586207  1752.322735  13.593150   
data analytics training_tablet  -8143.310345  2478.158577  -3.286033   

                                       P>|t|        [0.025        0.975]  
const                           3.595898e-36  20376.990484  27262.181930  
data analytics training_tablet  1.085473e-03 -13011.875906  -3274.744784  


In [92]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_leads', 
                                      baseline='data analytics training_tablet')

                                 Coef.   Std.Err.         t         P>|t|  \
const                        66.344828   7.860943  8.439806  3.252614e-16   
learn data analytics_mobile  42.517241  11.117052  3.824507  1.471279e-04   

                                [0.025     0.975]  
const                        50.901298  81.788357  
learn data analytics_mobile  20.676793  64.357690  


In [93]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_leads', 
                                      baseline='learn data analytics_mobile')

                                     Coef.   Std.Err.          t  \
const                           108.862069   7.860943  13.848475   
data analytics training_tablet  -42.517241  11.117052  -3.824507   

                                       P>|t|    [0.025      0.975]  
const                           2.731555e-37  93.41854  124.305598  
data analytics training_tablet  1.471279e-04 -64.35769  -20.676793  


In [94]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_conversions', 
                                      baseline='data analytics training_tablet')

                                 Coef.  Std.Err.         t         P>|t|  \
const                        21.931034  2.565308  8.549085  1.425494e-16   
learn data analytics_mobile  15.482759  3.627893  4.267700  2.352008e-05   

                                [0.025     0.975]  
const                        16.891256  26.970813  
learn data analytics_mobile   8.355436  22.610081  


In [95]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_conversions', 
                                      baseline='learn data analytics_mobile')

                                    Coef.  Std.Err.          t         P>|t|  \
const                           37.413793  2.565308  14.584523  1.442088e-40   
analytics for data_tablet      -12.013793  3.597534  -3.339452  9.003791e-04   
data analytics course_mobile   -13.680460  3.597534  -3.802733  1.602939e-04   
data analytics online_mobile   -11.780460  3.597534  -3.274593  1.129445e-03   
data analytics online_tablet   -11.896552  3.627893  -3.279190  1.111580e-03   
data analytics training_tablet -15.482759  3.627893  -4.267700  2.352008e-05   
learn data analytics_tablet    -11.380460  3.597534  -3.163406  1.651578e-03   

                                   [0.025     0.975]  
const                           32.374015  42.453571  
analytics for data_tablet      -19.081472  -4.946114  
data analytics course_mobile   -20.748139  -6.612781  
data analytics online_mobile   -18.848139  -4.712781  
data analytics online_tablet   -19.023875  -4.769229  
data analytics training_table

In [96]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_sales', 
                                      baseline='data analytics training_tablet')

                                   Coef.    Std.Err.         t         P>|t|  \
const                        4866.482759  559.397047  8.699515  4.522037e-17   
learn data analytics_mobile  2806.965517  791.106890  3.548150  4.234900e-04   

                                  [0.025       0.975]  
const                        3767.496915  5965.468603  
learn data analytics_mobile  1252.764832  4361.166203  


In [97]:
model = run_keyword_device_regression(keyw_dev_rollups, keyword_col='Keyword', device_col='Device', target_col='daily_sales', 
                                      baseline='learn data analytics_mobile')

                                      Coef.    Std.Err.          t  \
const                           7673.448276  559.397047  13.717356   
data analytics online_mobile   -2479.581609  784.486632  -3.160770   
data analytics training_tablet -2806.965517  791.106890  -3.548150   

                                       P>|t|       [0.025       0.975]  
const                           1.029071e-36  6574.462432  8772.434120  
data analytics online_mobile    1.666307e-03 -4020.776203  -938.387016  
data analytics training_tablet  4.234900e-04 -4361.166203 -1252.764832  


In [5]:
def run_subset_regression(dataframe, keyword, target_column, baseline=None):
    # Filter the dataframe to include only rows for the specified keyword
    subset = dataframe[dataframe['Keyword'] == keyword]
    
    # Check if there are at least 2 rows AND at least 2 unique devices for regression
    if subset.shape[0] < 2 or subset['Device'].nunique() < 2:
        print(f"Not enough data or device variation for keyword: {keyword}")
        return None
    
    # One-hot encode the Device column (create dummy variables for each device)
    dummies = pd.get_dummies(subset['Device'])
    
    # Optionally drop a baseline device column to avoid multicollinearity
    if baseline is not None and baseline in dummies.columns:
        dummies = dummies.drop(columns=baseline)
    
    # Add an intercept/constant column and ensure predictors are numeric
    X = sm.add_constant(dummies).astype(float)
    
    # Define the dependent variable (target) as numeric
    y = subset[target_column].astype(float)
    
    # Fit an Ordinary Least Squares (OLS) regression model
    model = sm.OLS(y, X).fit()
    
    # Print a header so each regression output is labeled by keyword
    print(f"\n--- Regression Summary for Keyword: '{keyword}' ---\n")
    
    # Print the full regression summary (coefficients, significance, R², etc.)
    print(model.summary())
    
    # Return the fitted model object for further inspection/use
    return model


In [66]:
mean_df

,Keyword,daily_spend,spend_share,daily_clicks,clicks_share,daily_impressions,impressions_share,daily_leads,leads_share,daily_conversions,conversions_share,daily_sales,sales_share
0,analytics for data,2683.014000,0.166180,1791.900000,0.165980,59558.500000,0.168510,261.700000,0.167970,82.133333,0.162953,18447.866667,0.166633
1,data analytics course,2690.498000,0.167313,1834.066667,0.169943,59033.866667,0.167947,262.466667,0.169347,85.066667,0.169090,18733.000000,0.170170
2,data analytics online,2513.527000,0.156827,1721.366667,0.160280,55709.100000,0.158763,249.966667,0.161133,77.566667,0.154050,17072.633333,0.155073
3,data analytics training,2511.675667,0.157197,1693.433333,0.158213,55855.666667,0.160163,236.600000,0.153247,80.966667,0.161230,17344.533333,0.159113
4,learn data analytics,2862.289667,0.176877,1845.466667,0.170497,60360.133333,0.171237,270.433333,0.174167,88.366667,0.173857,19545.033333,0.177590
5,online data analytics,2837.039667,0.175597,1899.066667,0.175090,61091.700000,0.173387,270.100000,0.174147,90.500000,0.178820,18857.800000,0.171433


In [7]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_spend', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.794
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0669
Time:                        14:07:23   Log-Likelihood:                -653.35
No. Observations:                  87   AIC:                             1313.
Df Residuals:                      84   BIC:                             1320.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [8]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_spend', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.794
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0669
Time:                        14:07:26   Log-Likelihood:                -653.35
No. Observations:                  87   AIC:                             1313.
Df Residuals:                      84   BIC:                             1320.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [9]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_clicks', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     2.541
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0848
Time:                        14:07:27   Log-Likelihood:                -621.77
No. Observations:                  87   AIC:                             1250.
Df Residuals:                      84   BIC:                             1257.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [10]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_clicks', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     2.541
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0848
Time:                        14:07:27   Log-Likelihood:                -621.77
No. Observations:                  87   AIC:                             1250.
Df Residuals:                      84   BIC:                             1257.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [11]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_impressions', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     2.948
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0579
Time:                        14:07:27   Log-Likelihood:                -922.88
No. Observations:                  87   AIC:                             1852.
Df Residuals:                      84   BIC:                             1859.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [12]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_impressions', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     2.948
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0579
Time:                        14:07:28   Log-Likelihood:                -922.88
No. Observations:                  87   AIC:                             1852.
Df Residuals:                      84   BIC:                             1859.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [13]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_leads', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     2.501
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0881
Time:                        14:07:28   Log-Likelihood:                -455.15
No. Observations:                  87   AIC:                             916.3
Df Residuals:                      84   BIC:                             923.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [14]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_leads', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     2.501
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0881
Time:                        14:07:28   Log-Likelihood:                -455.15
No. Observations:                  87   AIC:                             916.3
Df Residuals:                      84   BIC:                             923.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [15]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_conversions', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     3.968
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0226
Time:                        14:07:29   Log-Likelihood:                -353.88
No. Observations:                  87   AIC:                             713.8
Df Residuals:                      84   BIC:                             721.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [16]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_conversions', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     3.968
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0226
Time:                        14:07:29   Log-Likelihood:                -353.88
No. Observations:                  87   AIC:                             713.8
Df Residuals:                      84   BIC:                             721.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [17]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_sales', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.777
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0680
Time:                        14:07:29   Log-Likelihood:                -822.01
No. Observations:                  87   AIC:                             1650.
Df Residuals:                      84   BIC:                             1657.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [18]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics training', target_column='daily_sales', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.777
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0680
Time:                        14:07:30   Log-Likelihood:                -822.01
No. Observations:                  87   AIC:                             1650.
Df Residuals:                      84   BIC:                             1657.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [19]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_spend', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.195
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.308
Time:                        14:07:30   Log-Likelihood:                -662.63
No. Observations:                  88   AIC:                             1331.
Df Residuals:                      85   BIC:                             1339.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [20]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_spend', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.195
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.308
Time:                        14:07:30   Log-Likelihood:                -662.63
No. Observations:                  88   AIC:                             1331.
Df Residuals:                      85   BIC:                             1339.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [21]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_clicks', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9292
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.399
Time:                        14:07:30   Log-Likelihood:                -627.43
No. Observations:                  88   AIC:                             1261.
Df Residuals:                      85   BIC:                             1268.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [22]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_clicks', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9292
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.399
Time:                        14:07:31   Log-Likelihood:                -627.43
No. Observations:                  88   AIC:                             1261.
Df Residuals:                      85   BIC:                             1268.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [23]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_impressions', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.8474
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.432
Time:                        14:07:31   Log-Likelihood:                -934.62
No. Observations:                  88   AIC:                             1875.
Df Residuals:                      85   BIC:                             1883.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [24]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_impressions', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.8474
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.432
Time:                        14:07:31   Log-Likelihood:                -934.62
No. Observations:                  88   AIC:                             1875.
Df Residuals:                      85   BIC:                             1883.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [25]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_leads', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.8318
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.439
Time:                        14:07:31   Log-Likelihood:                -452.00
No. Observations:                  88   AIC:                             910.0
Df Residuals:                      85   BIC:                             917.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [26]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_leads', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.8318
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.439
Time:                        14:07:32   Log-Likelihood:                -452.00
No. Observations:                  88   AIC:                             910.0
Df Residuals:                      85   BIC:                             917.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [27]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_conversions', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7716
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.465
Time:                        14:07:32   Log-Likelihood:                -359.41
No. Observations:                  88   AIC:                             724.8
Df Residuals:                      85   BIC:                             732.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [28]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_conversions', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7716
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.465
Time:                        14:07:32   Log-Likelihood:                -359.41
No. Observations:                  88   AIC:                             724.8
Df Residuals:                      85   BIC:                             732.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [29]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_sales', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9205
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.402
Time:                        14:07:32   Log-Likelihood:                -836.24
No. Observations:                  88   AIC:                             1678.
Df Residuals:                      85   BIC:                             1686.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [30]:
model = run_subset_regression(keyw_dev_rollups, keyword='analytics for data', target_column='daily_sales', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9205
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.402
Time:                        14:07:32   Log-Likelihood:                -836.24
No. Observations:                  88   AIC:                             1678.
Df Residuals:                      85   BIC:                             1686.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [31]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_spend', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.588
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.210
Time:                        14:07:33   Log-Likelihood:                -671.14
No. Observations:                  90   AIC:                             1348.
Df Residuals:                      87   BIC:                             1356.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [32]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_spend', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.588
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.210
Time:                        14:07:33   Log-Likelihood:                -671.14
No. Observations:                  90   AIC:                             1348.
Df Residuals:                      87   BIC:                             1356.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [33]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_clicks', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     2.142
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.124
Time:                        14:07:33   Log-Likelihood:                -634.43
No. Observations:                  90   AIC:                             1275.
Df Residuals:                      87   BIC:                             1282.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [34]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_clicks', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     2.142
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.124
Time:                        14:07:33   Log-Likelihood:                -634.43
No. Observations:                  90   AIC:                             1275.
Df Residuals:                      87   BIC:                             1282.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [35]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_impressions', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1.342
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.267
Time:                        14:07:33   Log-Likelihood:                -948.39
No. Observations:                  90   AIC:                             1903.
Df Residuals:                      87   BIC:                             1910.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [36]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_impressions', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1.342
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.267
Time:                        14:07:34   Log-Likelihood:                -948.39
No. Observations:                  90   AIC:                             1903.
Df Residuals:                      87   BIC:                             1910.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [37]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_leads', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.438
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.243
Time:                        14:07:34   Log-Likelihood:                -462.53
No. Observations:                  90   AIC:                             931.1
Df Residuals:                      87   BIC:                             938.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [38]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_leads', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.438
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.243
Time:                        14:07:34   Log-Likelihood:                -462.53
No. Observations:                  90   AIC:                             931.1
Df Residuals:                      87   BIC:                             938.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [39]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_conversions', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     3.030
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0535
Time:                        14:07:34   Log-Likelihood:                -354.79
No. Observations:                  90   AIC:                             715.6
Df Residuals:                      87   BIC:                             723.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [40]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_conversions', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     3.030
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0535
Time:                        14:07:35   Log-Likelihood:                -354.79
No. Observations:                  90   AIC:                             715.6
Df Residuals:                      87   BIC:                             723.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [41]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_sales', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.711
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.187
Time:                        14:07:35   Log-Likelihood:                -845.08
No. Observations:                  90   AIC:                             1696.
Df Residuals:                      87   BIC:                             1704.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [42]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics course', target_column='daily_sales', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.711
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.187
Time:                        14:07:35   Log-Likelihood:                -845.08
No. Observations:                  90   AIC:                             1696.
Df Residuals:                      87   BIC:                             1704.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [43]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_spend', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.4415
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.645
Time:                        14:07:35   Log-Likelihood:                -655.21
No. Observations:                  89   AIC:                             1316.
Df Residuals:                      86   BIC:                             1324.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [44]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_spend', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.4415
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.645
Time:                        14:07:35   Log-Likelihood:                -655.21
No. Observations:                  89   AIC:                             1316.
Df Residuals:                      86   BIC:                             1324.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [45]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_clicks', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.4585
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.634
Time:                        14:07:35   Log-Likelihood:                -618.77
No. Observations:                  89   AIC:                             1244.
Df Residuals:                      86   BIC:                             1251.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [46]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_clicks', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.4585
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.634
Time:                        14:07:36   Log-Likelihood:                -618.77
No. Observations:                  89   AIC:                             1244.
Df Residuals:                      86   BIC:                             1251.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [47]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_impressions', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.021
Method:                 Least Squares   F-statistic:                    0.1022
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.903
Time:                        14:07:36   Log-Likelihood:                -928.46
No. Observations:                  89   AIC:                             1863.
Df Residuals:                      86   BIC:                             1870.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [48]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_impressions', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.021
Method:                 Least Squares   F-statistic:                    0.1022
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.903
Time:                        14:07:36   Log-Likelihood:                -928.46
No. Observations:                  89   AIC:                             1863.
Df Residuals:                      86   BIC:                             1870.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [49]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_leads', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.064
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.350
Time:                        14:07:36   Log-Likelihood:                -453.81
No. Observations:                  89   AIC:                             913.6
Df Residuals:                      86   BIC:                             921.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [50]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_leads', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.064
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.350
Time:                        14:07:36   Log-Likelihood:                -453.81
No. Observations:                  89   AIC:                             913.6
Df Residuals:                      86   BIC:                             921.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [51]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_conversions', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.1887
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.828
Time:                        14:07:37   Log-Likelihood:                -348.05
No. Observations:                  89   AIC:                             702.1
Df Residuals:                      86   BIC:                             709.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [52]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_conversions', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.1887
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.828
Time:                        14:07:37   Log-Likelihood:                -348.05
No. Observations:                  89   AIC:                             702.1
Df Residuals:                      86   BIC:                             709.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [53]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_sales', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9682
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.384
Time:                        14:07:37   Log-Likelihood:                -828.66
No. Observations:                  89   AIC:                             1663.
Df Residuals:                      86   BIC:                             1671.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [54]:
model = run_subset_regression(keyw_dev_rollups, keyword='data analytics online', target_column='daily_sales', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9682
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.384
Time:                        14:07:37   Log-Likelihood:                -828.66
No. Observations:                  89   AIC:                             1663.
Df Residuals:                      86   BIC:                             1671.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [55]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_spend', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     3.648
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0302
Time:                        14:07:37   Log-Likelihood:                -651.31
No. Observations:                  88   AIC:                             1309.
Df Residuals:                      85   BIC:                             1316.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [56]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_spend', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     3.648
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0302
Time:                        14:07:38   Log-Likelihood:                -651.31
No. Observations:                  88   AIC:                             1309.
Df Residuals:                      85   BIC:                             1316.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [57]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_clicks', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     2.933
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0587
Time:                        14:07:38   Log-Likelihood:                -619.40
No. Observations:                  88   AIC:                             1245.
Df Residuals:                      85   BIC:                             1252.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [58]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_clicks', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     2.933
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0587
Time:                        14:07:38   Log-Likelihood:                -619.40
No. Observations:                  88   AIC:                             1245.
Df Residuals:                      85   BIC:                             1252.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [59]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_impressions', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     3.144
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0482
Time:                        14:07:38   Log-Likelihood:                -919.63
No. Observations:                  88   AIC:                             1845.
Df Residuals:                      85   BIC:                             1853.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [60]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_impressions', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     3.144
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0482
Time:                        14:07:38   Log-Likelihood:                -919.63
No. Observations:                  88   AIC:                             1845.
Df Residuals:                      85   BIC:                             1853.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [61]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_leads', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     4.205
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0181
Time:                        14:07:38   Log-Likelihood:                -443.88
No. Observations:                  88   AIC:                             893.8
Df Residuals:                      85   BIC:                             901.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [62]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_leads', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     4.205
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0181
Time:                        14:07:39   Log-Likelihood:                -443.88
No. Observations:                  88   AIC:                             893.8
Df Residuals:                      85   BIC:                             901.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [63]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_conversions', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     6.637
Date:                Mon, 15 Sep 2025   Prob (F-statistic):            0.00210
Time:                        14:07:39   Log-Likelihood:                -350.42
No. Observations:                  88   AIC:                             706.8
Df Residuals:                      85   BIC:                             714.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [64]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_conversions', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     6.637
Date:                Mon, 15 Sep 2025   Prob (F-statistic):            0.00210
Time:                        14:07:39   Log-Likelihood:                -350.42
No. Observations:                  88   AIC:                             706.8
Df Residuals:                      85   BIC:                             714.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [65]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_sales', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     3.135
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0486
Time:                        14:07:39   Log-Likelihood:                -821.21
No. Observations:                  88   AIC:                             1648.
Df Residuals:                      85   BIC:                             1656.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [66]:
model = run_subset_regression(keyw_dev_rollups, keyword='learn data analytics', target_column='daily_sales', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     3.135
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0486
Time:                        14:07:39   Log-Likelihood:                -821.21
No. Observations:                  88   AIC:                             1648.
Df Residuals:                      85   BIC:                             1656.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [67]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_spend', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.6401
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.530
Time:                        14:07:40   Log-Likelihood:                -680.77
No. Observations:                  90   AIC:                             1368.
Df Residuals:                      87   BIC:                             1375.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [68]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_spend', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_spend   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.6401
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.530
Time:                        14:07:40   Log-Likelihood:                -680.77
No. Observations:                  90   AIC:                             1368.
Df Residuals:                      87   BIC:                             1375.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [69]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_clicks', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.314
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.274
Time:                        14:07:40   Log-Likelihood:                -645.03
No. Observations:                  90   AIC:                             1296.
Df Residuals:                      87   BIC:                             1304.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [70]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_clicks', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:           daily_clicks   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.314
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.274
Time:                        14:07:40   Log-Likelihood:                -645.03
No. Observations:                  90   AIC:                             1296.
Df Residuals:                      87   BIC:                             1304.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [71]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_impressions', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3821
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.684
Time:                        14:07:40   Log-Likelihood:                -957.33
No. Observations:                  90   AIC:                             1921.
Df Residuals:                      87   BIC:                             1928.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [72]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_impressions', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      daily_impressions   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3821
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.684
Time:                        14:07:40   Log-Likelihood:                -957.33
No. Observations:                  90   AIC:                             1921.
Df Residuals:                      87   BIC:                             1928.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [73]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_leads', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.7356
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.482
Time:                        14:07:41   Log-Likelihood:                -468.88
No. Observations:                  90   AIC:                             943.8
Df Residuals:                      87   BIC:                             951.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [74]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_leads', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_leads   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.7356
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.482
Time:                        14:07:41   Log-Likelihood:                -468.88
No. Observations:                  90   AIC:                             943.8
Df Residuals:                      87   BIC:                             951.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [75]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_conversions', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.023
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.364
Time:                        14:07:41   Log-Likelihood:                -372.49
No. Observations:                  90   AIC:                             751.0
Df Residuals:                      87   BIC:                             758.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [76]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_conversions', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      daily_conversions   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.023
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.364
Time:                        14:07:41   Log-Likelihood:                -372.49
No. Observations:                  90   AIC:                             751.0
Df Residuals:                      87   BIC:                             758.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [77]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_sales', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.657
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.197
Time:                        14:07:41   Log-Likelihood:                -851.56
No. Observations:                  90   AIC:                             1709.
Df Residuals:                      87   BIC:                             1717.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [78]:
model = run_subset_regression(keyw_dev_rollups, keyword='online data analytics', target_column='daily_sales', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            daily_sales   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.657
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.197
Time:                        14:07:41   Log-Likelihood:                -851.56
No. Observations:                  90   AIC:                             1709.
Df Residuals:                      87   BIC:                             1717.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [98]:
keyw_dev_eff= pd.read_csv(r"C:\Users\nehir\Desktop\data shit\ads project data\testable\keyw_device_efficiency.csv", delimiter=';', quotechar='"',
                            engine= 'python')
keyw_dev_eff

,Ad_Date,Keyword,Device,cost_per_click,cost_per_lead,ctr,cost_per_acquisition,cost_per_sale,profit_per_click,leads_per_conversion,lead_conv_rate,roi
0,2024-11-01,analytics for data,desktop,1.5512,10.6892,0.0337,30.2862,0.1462,9.0561,2.8333,0.1451,6.8379
1,2024-11-02,analytics for data,desktop,1.7882,11.8436,0.0275,36.6588,0.1843,7.9121,3.0952,0.1510,5.4245
2,2024-11-03,analytics for data,desktop,1.6349,9.5680,0.0244,32.3838,0.1537,9.0000,3.3846,0.1709,6.5049
3,2024-11-04,analytics for data,desktop,1.6397,12.7309,0.0317,36.5500,0.1528,9.0940,2.8710,0.1288,6.5460
4,2024-11-05,analytics for data,desktop,1.0938,7.3489,0.0418,29.6407,0.1173,8.2334,4.0333,0.1488,8.5277
...,...,...,...,...,...,...,...,...,...,...,...,...
527,2024-11-26,online data analytics,tablet,1.5404,10.2016,0.0269,28.4571,0.1152,11.8328,2.7895,0.1510,8.6816
528,2024-11-27,online data analytics,tablet,1.6996,16.9357,0.0318,23.7100,0.1673,8.4581,1.4000,0.1004,5.9764
529,2024-11-28,online data analytics,tablet,1.0064,9.5958,0.0292,19.6486,0.1113,8.0326,2.0476,0.1049,8.9816
530,2024-11-29,online data analytics,tablet,1.2700,11.3895,0.0269,25.8346,0.1125,10.0165,2.2683,0.1115,8.8867


In [99]:
mean_df = keyw_dev_eff.groupby(['Keyword', 'Device']).mean(numeric_only=True).reset_index()
mean_df

,Keyword,Device,cost_per_click,cost_per_lead,ctr,cost_per_acquisition,cost_per_sale,profit_per_click,leads_per_conversion,lead_conv_rate,roi
0,analytics for data,desktop,1.599821,11.625103,0.029793,33.918614,0.146921,9.427114,3.096769,0.141814,6.932893
1,analytics for data,mobile,1.518076,10.214986,0.031041,37.615414,0.153469,8.535031,3.825038,0.153086,6.674776
2,analytics for data,tablet,1.526850,9.766263,0.030317,33.255527,0.158080,8.877050,3.547153,0.160083,7.189047
3,data analytics course,desktop,1.431890,10.566387,0.032313,31.234773,0.138890,9.033357,3.014290,0.138577,7.441610
4,data analytics course,mobile,1.547073,10.833400,0.030573,35.408263,0.146093,9.212870,3.421043,0.149240,7.090993
5,data analytics course,tablet,1.540310,10.506970,0.031023,32.656823,0.152243,8.923843,3.155070,0.150293,6.810167
6,data analytics online,desktop,1.450200,10.243920,0.032280,33.669507,0.149880,8.481317,3.396873,0.145967,7.071387
7,data analytics online,mobile,1.471000,11.363337,0.030057,32.834050,0.162663,7.913963,3.010037,0.137570,6.418753
8,data analytics online,tablet,1.510583,10.257645,0.031486,33.906141,0.141507,9.448217,3.397500,0.152141,7.320769
9,data analytics training,desktop,1.484593,11.113290,0.028990,28.904424,0.145166,8.978779,2.757148,0.139538,7.346324


In [100]:
# List of numeric columns only
numeric_cols = mean_df.select_dtypes(include='number').columns

# Store results
extremes = []

for col in numeric_cols:
    max_val = mean_df[col].max()
    min_val = mean_df[col].min()
    
    # Get the row where max and min occur
    max_row = mean_df.loc[mean_df[col] == max_val].iloc[0]
    min_row = mean_df.loc[mean_df[col] == min_val].iloc[0]
    
    # Concatenate 'Keyword' and 'Device' for identification
    max_id = f"{max_row['Keyword']}_{max_row['Device']}"
    min_id = f"{min_row['Keyword']}_{min_row['Device']}"
    
    extremes.append({
        'column': col,
        'max_at': max_id,
        'min_at': min_id
    })

# Create result DataFrame
extremes_df = pd.DataFrame(extremes)

# Display
print(extremes_df)

                 column                          max_at  \
0        cost_per_click    learn data analytics_desktop   
1         cost_per_lead  data analytics training_mobile   
2                   ctr    online data analytics_mobile   
3  cost_per_acquisition       analytics for data_mobile   
4         cost_per_sale   online data analytics_desktop   
5      profit_per_click    learn data analytics_desktop   
6  leads_per_conversion       analytics for data_mobile   
7        lead_conv_rate       analytics for data_tablet   
8                   roi   data analytics course_desktop   

                            min_at  
0    data analytics course_desktop  
1        analytics for data_tablet  
2  data analytics training_desktop  
3  data analytics training_desktop  
4    data analytics course_desktop  
5     data analytics online_mobile  
6  data analytics training_desktop  
7   data analytics training_mobile  
8     data analytics online_mobile  


In [104]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='cost_per_click', 
                                      baseline='learn data analytics_desktop')

          Coef.  Std.Err.          t          P>|t|    [0.025    0.975]
const  1.621531  0.053658  30.219547  4.529046e-116  1.516114  1.726948


In [105]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='cost_per_click', 
                                      baseline='data analytics course_desktop')

         Coef.  Std.Err.          t          P>|t|    [0.025    0.975]
const  1.43189  0.052756  27.141508  2.589409e-101  1.328245  1.535535


In [106]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='cost_per_lead', 
                                      baseline='data analytics training_mobile')

                               Coef.  Std.Err.          t         P>|t|  \
const                      11.881697  0.474120  25.060531  3.713548e-91   
analytics for data_tablet  -2.115433  0.664896  -3.181601  1.553151e-03   

                              [0.025     0.975]  
const                      10.950245  12.813148  
analytics for data_tablet  -3.421681  -0.809186  


In [107]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='cost_per_click', 
                                      baseline='analytics for data_tablet')

         Coef.  Std.Err.          t          P>|t|    [0.025    0.975]
const  1.52685  0.052756  28.941477  5.475530e-110  1.423205  1.630495


In [108]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='ctr', 
                                      baseline='data analytics training_mobile')

          Coef.  Std.Err.          t          P>|t|    [0.025    0.975]
const  0.031817  0.001011  31.472975  5.764113e-122  0.029831  0.033803


In [109]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='ctr', 
                                      baseline='data analytics training_desktop')

         Coef.  Std.Err.          t          P>|t|    [0.025    0.975]
const  0.02899  0.001011  28.675984  1.024129e-108  0.027004  0.030976


In [110]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='cost_per_acquisition', 
                                      baseline='analytics for data_mobile')

                                     Coef.  Std.Err.          t         P>|t|  \
const                            37.615414  1.628118  23.103619  1.585971e-81   
data analytics training_desktop  -8.710990  2.302506  -3.783264  1.730001e-04   

                                   [0.025     0.975]  
const                            34.41683  40.813998  
data analytics training_desktop -13.23447  -4.187509  


In [111]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='cost_per_acquisition', 
                                      baseline='data analytics training_desktop')

                               Coef.  Std.Err.          t         P>|t|  \
const                      28.904424  1.628118  17.753276  2.398103e-55   
analytics for data_mobile   8.710990  2.302506   3.783264  1.730001e-04   

                              [0.025     0.975]  
const                      25.705840  32.103008  
analytics for data_mobile   4.187509  13.234470  


In [112]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='cost_per_sale', 
                                      baseline='online data analytics_desktop')

          Coef.  Std.Err.          t         P>|t|    [0.025    0.975]
const  0.166837  0.007055  23.646412  3.346113e-84  0.152976  0.180698


In [113]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='cost_per_sale', 
                                      baseline='data analytics course_desktop')

         Coef.  Std.Err.          t         P>|t|    [0.025    0.975]
const  0.13889  0.007055  19.685422  1.041950e-64  0.125029  0.152751


In [114]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='profit_per_click', 
                                      baseline='learn data analytics_desktop')

                                 Coef.  Std.Err.          t         P>|t|  \
const                         9.723062  0.407670  23.850311  3.311712e-85   
data analytics online_mobile -1.809099  0.571708  -3.164374  1.646197e-03   

                                [0.025     0.975]  
const                         8.922157  10.523967  
data analytics online_mobile -2.932271  -0.685927  


In [115]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='profit_per_click', 
                                      baseline='data analytics online_mobile')

                                 Coef.  Std.Err.          t         P>|t|  \
const                         7.913963  0.400818  19.744523  5.357626e-65   
learn data analytics_desktop  1.809099  0.571708   3.164374  1.646197e-03   

                                [0.025    0.975]  
const                         7.126520  8.701407  
learn data analytics_desktop  0.685927  2.932271  


In [116]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='leads_per_conversion', 
                                      baseline='analytics for data_mobile')

                                    Coef.  Std.Err.          t         P>|t|  \
const                            3.825038  0.180590  21.180752  4.788484e-72   
data analytics course_desktop   -0.810748  0.253256  -3.201298  1.452728e-03   
data analytics online_mobile    -0.815001  0.253256  -3.218093  1.371879e-03   
data analytics training_desktop -1.067890  0.255393  -4.181355  3.406639e-05   
data analytics training_mobile  -0.940476  0.255393  -3.682462  2.554432e-04   
learn data analytics_mobile     -0.839672  0.255393  -3.287763  1.078962e-03   
online data analytics_tablet    -0.801501  0.253256  -3.164787  1.643911e-03   

                                   [0.025    0.975]  
const                            3.470252  4.179824  
data analytics course_desktop   -1.308292 -0.313204  
data analytics online_mobile    -1.312545 -0.317457  
data analytics training_desktop -1.569633 -0.566147  
data analytics training_mobile  -1.442219 -0.438733  
learn data analytics_mobile     -1.

In [117]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='leads_per_conversion', 
                                      baseline='data analytics training_desktop')

                              Coef.  Std.Err.          t         P>|t|  \
const                      2.757148  0.180590  15.267424  1.141253e-43   
analytics for data_mobile  1.067890  0.255393   4.181355  3.406639e-05   
analytics for data_tablet  0.790005  0.253256   3.119393  1.913937e-03   

                             [0.025    0.975]  
const                      2.402362  3.111934  
analytics for data_mobile  0.566147  1.569633  
analytics for data_tablet  0.292461  1.287549  


In [118]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='lead_conv_rate', 
                                      baseline='analytics for data_tablet')

          Coef.  Std.Err.          t         P>|t|    [0.025    0.975]
const  0.160083  0.006053  26.448309  6.072287e-98  0.148192  0.171974


In [119]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='lead_conv_rate', 
                                      baseline='data analytics training_mobile')

          Coef.  Std.Err.          t         P>|t|    [0.025    0.975]
const  0.135641  0.006156  22.033443  3.009994e-76  0.123547  0.147736


In [120]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='roi', 
                                      baseline='data analytics course_desktop')

         Coef.  Std.Err.          t         P>|t|    [0.025    0.975]
const  7.44161  0.298794  24.905477  2.141696e-90  6.854602  8.028618


In [121]:
model = run_keyword_device_regression(keyw_dev_eff, keyword_col='Keyword', device_col='Device', target_col='roi', 
                                      baseline='data analytics online_mobile')

          Coef.  Std.Err.          t         P>|t|    [0.025    0.975]
const  6.418753  0.298794  21.482194  1.569327e-73  5.831745  7.005761


In [236]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='cost_per_click', baseline='desktop')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.5031
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.606
Time:                        18:15:33   Log-Likelihood:                -28.760
No. Observations:                  88   AIC:                             63.52
Df Residuals:                      85   BIC:                             70.95
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [128]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='cost_per_click', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.5031
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.606
Time:                        18:07:45   Log-Likelihood:                -28.760
No. Observations:                  88   AIC:                             63.52
Df Residuals:                      85   BIC:                             70.95
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [129]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='cost_per_lead', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     4.207
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0181
Time:                        18:07:46   Log-Likelihood:                -206.07
No. Observations:                  88   AIC:                             418.1
Df Residuals:                      85   BIC:                             425.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [131]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='cost_per_lead', baseline='desktop')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     4.207
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0181
Time:                        18:09:06   Log-Likelihood:                -206.07
No. Observations:                  88   AIC:                             418.1
Df Residuals:                      85   BIC:                             425.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [237]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='ctr', baseline='desktop')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3933
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.676
Time:                        18:15:52   Log-Likelihood:                 336.41
No. Observations:                  88   AIC:                            -666.8
Df Residuals:                      85   BIC:                            -659.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [133]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='ctr', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3933
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.676
Time:                        18:09:45   Log-Likelihood:                 336.41
No. Observations:                  88   AIC:                            -666.8
Df Residuals:                      85   BIC:                            -659.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [134]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='cost_per_acquisition', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.034
Model:                              OLS   Adj. R-squared:                  0.011
Method:                   Least Squares   F-statistic:                     1.484
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.233
Time:                          18:09:46   Log-Likelihood:                -329.68
No. Observations:                    88   AIC:                             665.4
Df Residuals:                        85   BIC:                             672.8
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

In [135]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='cost_per_acquisition', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.034
Model:                              OLS   Adj. R-squared:                  0.011
Method:                   Least Squares   F-statistic:                     1.484
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.233
Time:                          18:09:47   Log-Likelihood:                -329.68
No. Observations:                    88   AIC:                             665.4
Df Residuals:                        85   BIC:                             672.8
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

In [136]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='cost_per_sale', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.4435
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.643
Time:                        18:09:48   Log-Likelihood:                 148.23
No. Observations:                  88   AIC:                            -290.5
Df Residuals:                      85   BIC:                            -283.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [137]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='cost_per_sale', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.4435
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.643
Time:                        18:09:49   Log-Likelihood:                 148.23
No. Observations:                  88   AIC:                            -290.5
Df Residuals:                      85   BIC:                            -283.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [238]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='profit_per_click', baseline='desktop')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.308
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.276
Time:                        18:16:36   Log-Likelihood:                -189.45
No. Observations:                  88   AIC:                             384.9
Df Residuals:                      85   BIC:                             392.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [139]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='profit_per_click', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.308
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.276
Time:                        18:09:50   Log-Likelihood:                -189.45
No. Observations:                  88   AIC:                             384.9
Df Residuals:                      85   BIC:                             392.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [239]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='leads_per_conversion', baseline='desktop')


--- Regression Summary for Keyword: 'analytics for data' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.046
Model:                              OLS   Adj. R-squared:                  0.024
Method:                   Least Squares   F-statistic:                     2.072
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.132
Time:                          18:16:53   Log-Likelihood:                -151.37
No. Observations:                    88   AIC:                             308.7
Df Residuals:                        85   BIC:                             316.2
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

In [141]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='leads_per_conversion', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.046
Model:                              OLS   Adj. R-squared:                  0.024
Method:                   Least Squares   F-statistic:                     2.072
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.132
Time:                          18:09:51   Log-Likelihood:                -151.37
No. Observations:                    88   AIC:                             308.7
Df Residuals:                        85   BIC:                             316.2
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

In [142]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='lead_conv_rate', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     1.969
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.146
Time:                        18:10:20   Log-Likelihood:                 170.10
No. Observations:                  88   AIC:                            -334.2
Df Residuals:                      85   BIC:                            -326.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [240]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='lead_conv_rate', baseline='desktop')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     1.969
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.146
Time:                        18:17:57   Log-Likelihood:                 170.10
No. Observations:                  88   AIC:                            -334.2
Df Residuals:                      85   BIC:                            -326.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [144]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='roi', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.8045
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.451
Time:                        18:10:23   Log-Likelihood:                -162.30
No. Observations:                  88   AIC:                             330.6
Df Residuals:                      85   BIC:                             338.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [145]:
model = run_subset_regression(keyw_dev_eff, keyword='analytics for data', target_column='roi', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.8045
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.451
Time:                        18:10:24   Log-Likelihood:                -162.30
No. Observations:                  88   AIC:                             330.6
Df Residuals:                      85   BIC:                             338.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [241]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='cost_per_click', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.425
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.246
Time:                        18:18:18   Log-Likelihood:                -16.785
No. Observations:                  90   AIC:                             39.57
Df Residuals:                      87   BIC:                             47.07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [147]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='cost_per_lead', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.1494
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.861
Time:                        18:10:26   Log-Likelihood:                -207.32
No. Observations:                  90   AIC:                             420.6
Df Residuals:                      87   BIC:                             428.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [242]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='ctr', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9000
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.410
Time:                        18:18:37   Log-Likelihood:                 346.89
No. Observations:                  90   AIC:                            -687.8
Df Residuals:                      87   BIC:                            -680.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [243]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='cost_per_acquisition', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.033
Model:                              OLS   Adj. R-squared:                  0.011
Method:                   Least Squares   F-statistic:                     1.482
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.233
Time:                          18:18:59   Log-Likelihood:                -329.24
No. Observations:                    90   AIC:                             664.5
Df Residuals:                        87   BIC:                             672.0
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [150]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='cost_per_sale', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.737
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.182
Time:                        18:10:28   Log-Likelihood:                 196.35
No. Observations:                  90   AIC:                            -386.7
Df Residuals:                      87   BIC:                            -379.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [151]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='profit_per_click', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.1342
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.875
Time:                        18:10:29   Log-Likelihood:                -196.38
No. Observations:                  90   AIC:                             398.8
Df Residuals:                      87   BIC:                             406.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [244]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='leads_per_conversion', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.028
Model:                              OLS   Adj. R-squared:                  0.005
Method:                   Least Squares   F-statistic:                     1.244
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.293
Time:                          18:19:25   Log-Likelihood:                -127.47
No. Observations:                    90   AIC:                             260.9
Df Residuals:                        87   BIC:                             268.4
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [153]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='lead_conv_rate', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.077
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.345
Time:                        18:10:30   Log-Likelihood:                 177.61
No. Observations:                  90   AIC:                            -349.2
Df Residuals:                      87   BIC:                            -341.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [154]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='roi', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.458
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.238
Time:                        18:10:31   Log-Likelihood:                -158.68
No. Observations:                  90   AIC:                             323.4
Df Residuals:                      87   BIC:                             330.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [155]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='cost_per_click', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.425
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.246
Time:                        18:10:32   Log-Likelihood:                -16.785
No. Observations:                  90   AIC:                             39.57
Df Residuals:                      87   BIC:                             47.07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [156]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='cost_per_lead', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.1494
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.861
Time:                        18:10:32   Log-Likelihood:                -207.32
No. Observations:                  90   AIC:                             420.6
Df Residuals:                      87   BIC:                             428.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [157]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='ctr', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9000
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.410
Time:                        18:10:33   Log-Likelihood:                 346.89
No. Observations:                  90   AIC:                            -687.8
Df Residuals:                      87   BIC:                            -680.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [158]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='cost_per_acquisition', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.033
Model:                              OLS   Adj. R-squared:                  0.011
Method:                   Least Squares   F-statistic:                     1.482
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.233
Time:                          18:10:34   Log-Likelihood:                -329.24
No. Observations:                    90   AIC:                             664.5
Df Residuals:                        87   BIC:                             672.0
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [245]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='cost_per_sale', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.737
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.182
Time:                        18:19:44   Log-Likelihood:                 196.35
No. Observations:                  90   AIC:                            -386.7
Df Residuals:                      87   BIC:                            -379.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [160]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='profit_per_click', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.1342
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.875
Time:                        18:10:36   Log-Likelihood:                -196.38
No. Observations:                  90   AIC:                             398.8
Df Residuals:                      87   BIC:                             406.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [161]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='leads_per_conversion', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.028
Model:                              OLS   Adj. R-squared:                  0.005
Method:                   Least Squares   F-statistic:                     1.244
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.293
Time:                          18:10:36   Log-Likelihood:                -127.47
No. Observations:                    90   AIC:                             260.9
Df Residuals:                        87   BIC:                             268.4
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [246]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='lead_conv_rate', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.077
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.345
Time:                        18:19:57   Log-Likelihood:                 177.61
No. Observations:                  90   AIC:                            -349.2
Df Residuals:                      87   BIC:                            -341.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [247]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics course', target_column='roi', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.458
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.238
Time:                        18:20:06   Log-Likelihood:                -158.68
No. Observations:                  90   AIC:                             323.4
Df Residuals:                      87   BIC:                             330.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [248]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='cost_per_click', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3870
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.680
Time:                        18:20:24   Log-Likelihood:                -7.3535
No. Observations:                  89   AIC:                             20.71
Df Residuals:                      86   BIC:                             28.17
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [165]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='cost_per_lead', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.604
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.207
Time:                        18:10:39   Log-Likelihood:                -215.44
No. Observations:                  89   AIC:                             436.9
Df Residuals:                      86   BIC:                             444.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [166]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='ctr', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.124
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.330
Time:                        18:10:40   Log-Likelihood:                 333.27
No. Observations:                  89   AIC:                            -660.5
Df Residuals:                      86   BIC:                            -653.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [167]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='cost_per_acquisition', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.003
Model:                              OLS   Adj. R-squared:                 -0.020
Method:                   Least Squares   F-statistic:                    0.1291
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.879
Time:                          18:10:41   Log-Likelihood:                -315.65
No. Observations:                    89   AIC:                             637.3
Df Residuals:                        86   BIC:                             644.8
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [168]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='cost_per_sale', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.177
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.313
Time:                        18:10:41   Log-Likelihood:                 136.00
No. Observations:                  89   AIC:                            -266.0
Df Residuals:                      86   BIC:                            -258.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [169]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='profit_per_click', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     2.921
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0592
Time:                        18:10:42   Log-Likelihood:                -204.94
No. Observations:                  89   AIC:                             415.9
Df Residuals:                      86   BIC:                             423.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [170]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='leads_per_conversion', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.048
Model:                              OLS   Adj. R-squared:                  0.026
Method:                   Least Squares   F-statistic:                     2.191
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.118
Time:                          18:11:14   Log-Likelihood:                -107.62
No. Observations:                    89   AIC:                             221.2
Df Residuals:                        86   BIC:                             228.7
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [171]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='lead_conv_rate', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.423
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.247
Time:                        18:11:16   Log-Likelihood:                 177.96
No. Observations:                  89   AIC:                            -349.9
Df Residuals:                      86   BIC:                            -342.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [172]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='roi', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.637
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.200
Time:                        18:11:16   Log-Likelihood:                -185.66
No. Observations:                  89   AIC:                             377.3
Df Residuals:                      86   BIC:                             384.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [173]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='cost_per_click', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3870
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.680
Time:                        18:11:17   Log-Likelihood:                -7.3535
No. Observations:                  89   AIC:                             20.71
Df Residuals:                      86   BIC:                             28.17
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [249]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='cost_per_lead', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.604
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.207
Time:                        18:21:22   Log-Likelihood:                -215.44
No. Observations:                  89   AIC:                             436.9
Df Residuals:                      86   BIC:                             444.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [250]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='ctr', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.124
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.330
Time:                        18:21:30   Log-Likelihood:                 333.27
No. Observations:                  89   AIC:                            -660.5
Df Residuals:                      86   BIC:                            -653.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [176]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='cost_per_acquisition', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.003
Model:                              OLS   Adj. R-squared:                 -0.020
Method:                   Least Squares   F-statistic:                    0.1291
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.879
Time:                          18:11:19   Log-Likelihood:                -315.65
No. Observations:                    89   AIC:                             637.3
Df Residuals:                        86   BIC:                             644.8
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [177]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='cost_per_sale', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.177
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.313
Time:                        18:11:20   Log-Likelihood:                 136.00
No. Observations:                  89   AIC:                            -266.0
Df Residuals:                      86   BIC:                            -258.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [178]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='profit_per_click', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     2.921
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0592
Time:                        18:11:21   Log-Likelihood:                -204.94
No. Observations:                  89   AIC:                             415.9
Df Residuals:                      86   BIC:                             423.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [179]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='leads_per_conversion', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.048
Model:                              OLS   Adj. R-squared:                  0.026
Method:                   Least Squares   F-statistic:                     2.191
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.118
Time:                          18:11:49   Log-Likelihood:                -107.62
No. Observations:                    89   AIC:                             221.2
Df Residuals:                        86   BIC:                             228.7
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [180]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='lead_conv_rate', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.423
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.247
Time:                        18:11:50   Log-Likelihood:                 177.96
No. Observations:                  89   AIC:                            -349.9
Df Residuals:                      86   BIC:                            -342.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [181]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics online', target_column='roi', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.637
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.200
Time:                        18:11:51   Log-Likelihood:                -185.66
No. Observations:                  89   AIC:                             377.3
Df Residuals:                      86   BIC:                             384.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [182]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='cost_per_click', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                    0.2822
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.755
Time:                        18:11:52   Log-Likelihood:                -19.928
No. Observations:                  87   AIC:                             45.86
Df Residuals:                      84   BIC:                             53.25
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [251]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='cost_per_lead', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3970
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.674
Time:                        18:21:53   Log-Likelihood:                -226.40
No. Observations:                  87   AIC:                             458.8
Df Residuals:                      84   BIC:                             466.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [252]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='ctr', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.295
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.107
Time:                        18:22:02   Log-Likelihood:                 329.12
No. Observations:                  87   AIC:                            -652.2
Df Residuals:                      84   BIC:                            -644.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [185]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='cost_per_acquisition', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.093
Model:                              OLS   Adj. R-squared:                  0.072
Method:                   Least Squares   F-statistic:                     4.312
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):             0.0165
Time:                          18:11:54   Log-Likelihood:                -294.44
No. Observations:                    87   AIC:                             594.9
Df Residuals:                        84   BIC:                             602.3
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

In [253]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='cost_per_sale', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.1667
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.847
Time:                        18:22:19   Log-Likelihood:                 184.08
No. Observations:                  87   AIC:                            -362.2
Df Residuals:                      84   BIC:                            -354.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [187]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='profit_per_click', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                   0.07916
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.924
Time:                        18:13:32   Log-Likelihood:                -185.14
No. Observations:                  87   AIC:                             376.3
Df Residuals:                      84   BIC:                             383.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [188]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='leads_per_conversion', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.033
Model:                              OLS   Adj. R-squared:                  0.010
Method:                   Least Squares   F-statistic:                     1.449
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.241
Time:                          18:13:33   Log-Likelihood:                -101.38
No. Observations:                    87   AIC:                             208.8
Df Residuals:                        84   BIC:                             216.2
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

In [189]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='lead_conv_rate', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.1379
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.871
Time:                        18:13:34   Log-Likelihood:                 171.45
No. Observations:                  87   AIC:                            -336.9
Df Residuals:                      84   BIC:                            -329.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [254]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='roi', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7044
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.497
Time:                        18:22:41   Log-Likelihood:                -164.83
No. Observations:                  87   AIC:                             335.7
Df Residuals:                      84   BIC:                             343.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [255]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='cost_per_click', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                    0.2822
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.755
Time:                        18:22:49   Log-Likelihood:                -19.928
No. Observations:                  87   AIC:                             45.86
Df Residuals:                      84   BIC:                             53.25
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [192]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='cost_per_lead', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3970
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.674
Time:                        18:13:35   Log-Likelihood:                -226.40
No. Observations:                  87   AIC:                             458.8
Df Residuals:                      84   BIC:                             466.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [193]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='ctr', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.295
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.107
Time:                        18:13:36   Log-Likelihood:                 329.12
No. Observations:                  87   AIC:                            -652.2
Df Residuals:                      84   BIC:                            -644.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [256]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='cost_per_acquisition', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.093
Model:                              OLS   Adj. R-squared:                  0.072
Method:                   Least Squares   F-statistic:                     4.312
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):             0.0165
Time:                          18:23:06   Log-Likelihood:                -294.44
No. Observations:                    87   AIC:                             594.9
Df Residuals:                        84   BIC:                             602.3
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

In [195]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='cost_per_sale', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.1667
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.847
Time:                        18:13:38   Log-Likelihood:                 184.08
No. Observations:                  87   AIC:                            -362.2
Df Residuals:                      84   BIC:                            -354.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [196]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='profit_per_click', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                   0.07916
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.924
Time:                        18:13:39   Log-Likelihood:                -185.14
No. Observations:                  87   AIC:                             376.3
Df Residuals:                      84   BIC:                             383.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [257]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='leads_per_conversion', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.033
Model:                              OLS   Adj. R-squared:                  0.010
Method:                   Least Squares   F-statistic:                     1.449
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.241
Time:                          18:23:43   Log-Likelihood:                -101.38
No. Observations:                    87   AIC:                             208.8
Df Residuals:                        84   BIC:                             216.2
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

In [198]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='lead_conv_rate', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.1379
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.871
Time:                        18:13:40   Log-Likelihood:                 171.45
No. Observations:                  87   AIC:                            -336.9
Df Residuals:                      84   BIC:                            -329.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [199]:
model = run_subset_regression(keyw_dev_eff, keyword='data analytics training', target_column='roi', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7044
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.497
Time:                        18:13:40   Log-Likelihood:                -164.83
No. Observations:                  87   AIC:                             335.7
Df Residuals:                      84   BIC:                             343.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [258]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='cost_per_click', baseline='desktop')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.6565
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.521
Time:                        18:23:53   Log-Likelihood:                -4.9267
No. Observations:                  88   AIC:                             15.85
Df Residuals:                      85   BIC:                             23.29
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [201]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='cost_per_lead', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.1381
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.871
Time:                        18:13:41   Log-Likelihood:                -187.33
No. Observations:                  88   AIC:                             380.7
Df Residuals:                      85   BIC:                             388.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [259]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='ctr', baseline='desktop')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.021
Method:                 Least Squares   F-statistic:                   0.09786
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.907
Time:                        18:24:03   Log-Likelihood:                 350.21
No. Observations:                  88   AIC:                            -694.4
Df Residuals:                      85   BIC:                            -687.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [203]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='cost_per_acquisition', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.022
Model:                              OLS   Adj. R-squared:                 -0.001
Method:                   Least Squares   F-statistic:                    0.9606
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.387
Time:                          18:13:42   Log-Likelihood:                -320.40
No. Observations:                    88   AIC:                             646.8
Df Residuals:                        85   BIC:                             654.2
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

In [260]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='cost_per_sale', baseline='desktop')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7032
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.498
Time:                        18:24:13   Log-Likelihood:                 177.08
No. Observations:                  88   AIC:                            -348.2
Df Residuals:                      85   BIC:                            -340.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [261]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='profit_per_click', baseline='desktop')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.5595
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.574
Time:                        18:24:21   Log-Likelihood:                -198.98
No. Observations:                  88   AIC:                             404.0
Df Residuals:                      85   BIC:                             411.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [206]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='leads_per_conversion', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.046
Model:                              OLS   Adj. R-squared:                  0.023
Method:                   Least Squares   F-statistic:                     2.047
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.135
Time:                          18:13:44   Log-Likelihood:                -110.48
No. Observations:                    88   AIC:                             227.0
Df Residuals:                        85   BIC:                             234.4
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

In [262]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='lead_conv_rate', baseline='desktop')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                    0.2775
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.758
Time:                        18:24:37   Log-Likelihood:                 188.97
No. Observations:                  88   AIC:                            -371.9
Df Residuals:                      85   BIC:                            -364.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [208]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='roi', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                   0.06999
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.932
Time:                        18:13:45   Log-Likelihood:                -171.71
No. Observations:                  88   AIC:                             349.4
Df Residuals:                      85   BIC:                             356.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [209]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='cost_per_click', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.6565
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.521
Time:                        18:13:46   Log-Likelihood:                -4.9267
No. Observations:                  88   AIC:                             15.85
Df Residuals:                      85   BIC:                             23.29
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [210]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='cost_per_lead', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.1381
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.871
Time:                        18:13:47   Log-Likelihood:                -187.33
No. Observations:                  88   AIC:                             380.7
Df Residuals:                      85   BIC:                             388.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [211]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='ctr', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.021
Method:                 Least Squares   F-statistic:                   0.09786
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.907
Time:                        18:13:47   Log-Likelihood:                 350.21
No. Observations:                  88   AIC:                            -694.4
Df Residuals:                      85   BIC:                            -687.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [263]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='cost_per_acquisition', baseline='desktop')


--- Regression Summary for Keyword: 'learn data analytics' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.022
Model:                              OLS   Adj. R-squared:                 -0.001
Method:                   Least Squares   F-statistic:                    0.9606
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.387
Time:                          18:24:55   Log-Likelihood:                -320.40
No. Observations:                    88   AIC:                             646.8
Df Residuals:                        85   BIC:                             654.2
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

In [213]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='cost_per_sale', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7032
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.498
Time:                        18:13:48   Log-Likelihood:                 177.08
No. Observations:                  88   AIC:                            -348.2
Df Residuals:                      85   BIC:                            -340.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [214]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='profit_per_click', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.5595
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.574
Time:                        18:13:49   Log-Likelihood:                -198.98
No. Observations:                  88   AIC:                             404.0
Df Residuals:                      85   BIC:                             411.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [215]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='leads_per_conversion', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.046
Model:                              OLS   Adj. R-squared:                  0.023
Method:                   Least Squares   F-statistic:                     2.047
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.135
Time:                          18:13:49   Log-Likelihood:                -110.48
No. Observations:                    88   AIC:                             227.0
Df Residuals:                        85   BIC:                             234.4
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

In [216]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='lead_conv_rate', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                    0.2775
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.758
Time:                        18:13:50   Log-Likelihood:                 188.97
No. Observations:                  88   AIC:                            -371.9
Df Residuals:                      85   BIC:                            -364.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [217]:
model = run_subset_regression(keyw_dev_eff, keyword='learn data analytics', target_column='roi', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                   0.06999
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.932
Time:                        18:13:51   Log-Likelihood:                -171.71
No. Observations:                  88   AIC:                             349.4
Df Residuals:                      85   BIC:                             356.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [218]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='cost_per_click', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.5005
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.608
Time:                        18:13:51   Log-Likelihood:                -1.0703
No. Observations:                  90   AIC:                             8.141
Df Residuals:                      87   BIC:                             15.64
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [219]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='cost_per_lead', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.039
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.358
Time:                        18:13:52   Log-Likelihood:                -183.93
No. Observations:                  90   AIC:                             373.9
Df Residuals:                      87   BIC:                             381.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [264]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='ctr', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9168
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.404
Time:                        18:25:14   Log-Likelihood:                 335.39
No. Observations:                  90   AIC:                            -664.8
Df Residuals:                      87   BIC:                            -657.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [221]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='cost_per_acquisition', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.002
Model:                              OLS   Adj. R-squared:                 -0.021
Method:                   Least Squares   F-statistic:                   0.07184
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.931
Time:                          18:13:53   Log-Likelihood:                -299.99
No. Observations:                    90   AIC:                             606.0
Df Residuals:                        87   BIC:                             613.5
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [222]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='cost_per_sale', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     2.736
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0704
Time:                        18:13:54   Log-Likelihood:                 173.41
No. Observations:                  90   AIC:                            -340.8
Df Residuals:                      87   BIC:                            -333.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [223]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='profit_per_click', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.018
Method:                 Least Squares   F-statistic:                    0.1930
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.825
Time:                        18:14:24   Log-Likelihood:                -185.72
No. Observations:                  90   AIC:                             377.4
Df Residuals:                      87   BIC:                             384.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [224]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='leads_per_conversion', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.011
Model:                              OLS   Adj. R-squared:                 -0.012
Method:                   Least Squares   F-statistic:                    0.4796
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.621
Time:                          18:14:29   Log-Likelihood:                -110.57
No. Observations:                    90   AIC:                             227.1
Df Residuals:                        87   BIC:                             234.6
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [225]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='lead_conv_rate', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     1.920
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.153
Time:                        18:14:30   Log-Likelihood:                 182.90
No. Observations:                  90   AIC:                            -359.8
Df Residuals:                      87   BIC:                            -352.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [226]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='roi', baseline='tablet')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9290
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.399
Time:                        18:14:31   Log-Likelihood:                -157.14
No. Observations:                  90   AIC:                             320.3
Df Residuals:                      87   BIC:                             327.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [265]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='cost_per_click', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:         cost_per_click   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.5005
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.608
Time:                        18:25:35   Log-Likelihood:                -1.0703
No. Observations:                  90   AIC:                             8.141
Df Residuals:                      87   BIC:                             15.64
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [228]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='cost_per_lead', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_lead   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.039
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.358
Time:                        18:14:32   Log-Likelihood:                -183.93
No. Observations:                  90   AIC:                             373.9
Df Residuals:                      87   BIC:                             381.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [229]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='ctr', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:                    ctr   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9168
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.404
Time:                        18:14:33   Log-Likelihood:                 335.39
No. Observations:                  90   AIC:                            -664.8
Df Residuals:                      87   BIC:                            -657.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [266]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='cost_per_acquisition', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                             OLS Regression Results                             
Dep. Variable:     cost_per_acquisition   R-squared:                       0.002
Model:                              OLS   Adj. R-squared:                 -0.021
Method:                   Least Squares   F-statistic:                   0.07184
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.931
Time:                          18:25:44   Log-Likelihood:                -299.99
No. Observations:                    90   AIC:                             606.0
Df Residuals:                        87   BIC:                             613.5
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [267]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='cost_per_sale', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:          cost_per_sale   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     2.736
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0704
Time:                        18:25:51   Log-Likelihood:                 173.41
No. Observations:                  90   AIC:                            -340.8
Df Residuals:                      87   BIC:                            -333.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [268]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='profit_per_click', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:       profit_per_click   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.018
Method:                 Least Squares   F-statistic:                    0.1930
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.825
Time:                        18:26:21   Log-Likelihood:                -185.72
No. Observations:                  90   AIC:                             377.4
Df Residuals:                      87   BIC:                             384.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [233]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='leads_per_conversion', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                             OLS Regression Results                             
Dep. Variable:     leads_per_conversion   R-squared:                       0.011
Model:                              OLS   Adj. R-squared:                 -0.012
Method:                   Least Squares   F-statistic:                    0.4796
Date:                  Mon, 15 Sep 2025   Prob (F-statistic):              0.621
Time:                          18:14:36   Log-Likelihood:                -110.57
No. Observations:                    90   AIC:                             227.1
Df Residuals:                        87   BIC:                             234.6
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [269]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='lead_conv_rate', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:         lead_conv_rate   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     1.920
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.153
Time:                        18:26:31   Log-Likelihood:                 182.90
No. Observations:                  90   AIC:                            -359.8
Df Residuals:                      87   BIC:                            -352.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [270]:
model = run_subset_regression(keyw_dev_eff, keyword='online data analytics', target_column='roi', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:                    roi   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9290
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.399
Time:                        18:26:39   Log-Likelihood:                -157.14
No. Observations:                  90   AIC:                             320.3
Df Residuals:                      87   BIC:                             327.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [272]:
keyw_dev_shares= pd.read_csv(r"C:\Users\nehir\Desktop\data shit\ads project data\testable\keyw_dev_shares.csv", delimiter=';', quotechar='"',
                            engine= 'python')
keyw_dev_shares

,Ad_Date,Keyword,Device,daily_spend,spend_share,daily_clicks,clicks_share,daily_impressions,impressions_share,daily_leads,leads_share,daily_conversions,conversions_share,daily_sales,sales_share
0,2024-11-01,analytics for data,desktop,1272.02,0.0877,820.0,0.0868,24359.0,0.0829,119.0,0.0884,42.0,0.0972,8698.0,0.0931
1,2024-11-02,analytics for data,desktop,1539.67,0.1033,861.0,0.0896,31313.0,0.0925,130.0,0.0890,42.0,0.0850,8352.0,0.0786
2,2024-11-03,analytics for data,desktop,841.98,0.0563,515.0,0.0504,21069.0,0.0618,88.0,0.0589,26.0,0.0575,5477.0,0.0536
3,2024-11-04,analytics for data,desktop,1133.05,0.0696,691.0,0.0606,21814.0,0.0635,89.0,0.0558,31.0,0.0642,7417.0,0.0656
4,2024-11-05,analytics for data,desktop,889.22,0.0518,813.0,0.0670,19470.0,0.0512,121.0,0.0712,30.0,0.0569,7583.0,0.0617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,2024-11-26,online data analytics,tablet,1081.37,0.0612,702.0,0.0590,26089.0,0.0669,106.0,0.0610,38.0,0.0654,9388.0,0.0788
528,2024-11-27,online data analytics,tablet,474.20,0.0298,279.0,0.0284,8766.0,0.0262,28.0,0.0204,20.0,0.0396,2834.0,0.0270
529,2024-11-28,online data analytics,tablet,412.62,0.0307,410.0,0.0430,14065.0,0.0480,43.0,0.0340,21.0,0.0488,3706.0,0.0385
530,2024-11-29,online data analytics,tablet,1059.22,0.0694,834.0,0.0803,31006.0,0.0889,93.0,0.0644,41.0,0.0827,9413.0,0.0895


In [276]:
mean_df = keyw_dev_shares.groupby(['Keyword', 'Device']).mean(numeric_only=True).reset_index()
mean_df

,Keyword,Device,daily_spend,spend_share,daily_clicks,clicks_share,daily_impressions,impressions_share,daily_leads,leads_share,daily_conversions,conversions_share,daily_sales,sales_share
0,analytics for data,desktop,956.325172,0.059048,617.241379,0.057069,20974.172414,0.059138,87.206897,0.055928,29.965517,0.059828,6623.344828,0.059555
1,analytics for data,mobile,980.791034,0.060531,662.965517,0.061321,21612.827586,0.061052,97.137931,0.062048,28.724138,0.056655,6640.862069,0.059755
2,analytics for data,tablet,810.468333,0.050587,554.366667,0.051547,18391.066667,0.052343,83.500000,0.053930,25.400000,0.050373,5625.800000,0.051300
3,data analytics course,desktop,950.776333,0.059510,666.633333,0.062117,20865.066667,0.059727,90.966667,0.059340,31.133333,0.062533,6871.966667,0.062940
4,data analytics course,mobile,783.619667,0.048500,525.000000,0.048493,17412.500000,0.049250,77.066667,0.049277,23.733333,0.046873,5484.366667,0.049307
5,data analytics course,tablet,956.102000,0.059300,642.433333,0.059327,20756.300000,0.058977,94.433333,0.060737,30.200000,0.059687,6376.666667,0.057897
6,data analytics online,desktop,898.817333,0.056650,615.333333,0.057560,19332.100000,0.055457,90.800000,0.059000,27.266667,0.054893,6081.433333,0.055873
7,data analytics online,mobile,806.789333,0.051073,553.333333,0.052323,18418.000000,0.053317,75.933333,0.049887,25.633333,0.051273,5193.866667,0.047653
8,data analytics online,tablet,835.779655,0.050803,571.758621,0.052131,18578.275862,0.051707,86.103448,0.054031,25.517241,0.049545,5997.241379,0.053328
9,data analytics training,desktop,952.045862,0.059431,643.517241,0.059855,21723.517241,0.062052,92.172414,0.059521,32.103448,0.063721,6595.068966,0.060300


In [277]:
numeric_cols = mean_df.select_dtypes(include='number').columns

# Store results
extremes = []

for col in numeric_cols:
    max_val = mean_df[col].max()
    min_val = mean_df[col].min()
    
    # Get the row where max and min occur
    max_row = mean_df.loc[mean_df[col] == max_val].iloc[0]
    min_row = mean_df.loc[mean_df[col] == min_val].iloc[0]
    
    # Concatenate 'Keyword' and 'Device' for identification
    max_id = f"{max_row['Keyword']}_{max_row['Device']}"
    min_id = f"{min_row['Keyword']}_{min_row['Device']}"
    
    extremes.append({
        'column': col,
        'max_at': max_id,
        'min_at': min_id
    })

# Create result DataFrame
extremes_df = pd.DataFrame(extremes)

# Display
print(extremes_df)

               column                       max_at  \
0         daily_spend  learn data analytics_mobile   
1         spend_share  learn data analytics_mobile   
2        daily_clicks  learn data analytics_mobile   
3        clicks_share  learn data analytics_mobile   
4   daily_impressions  learn data analytics_mobile   
5   impressions_share  learn data analytics_mobile   
6         daily_leads  learn data analytics_mobile   
7         leads_share  learn data analytics_mobile   
8   daily_conversions  learn data analytics_mobile   
9   conversions_share  learn data analytics_mobile   
10        daily_sales  learn data analytics_mobile   
11        sales_share  learn data analytics_mobile   

                            min_at  
0   data analytics training_tablet  
1   data analytics training_tablet  
2   data analytics training_tablet  
3   data analytics training_tablet  
4   data analytics training_tablet  
5   data analytics training_tablet  
6   data analytics training_tablet  
7

In [278]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='spend_share', 
                                      baseline='learn data analytics_mobile')

                                   Coef.  Std.Err.          t         P>|t|  \
const                           0.070345  0.004922  14.291630  2.963371e-39   
data analytics course_mobile   -0.021845  0.006903  -3.164702  1.644380e-03   
data analytics training_tablet -0.026355  0.006961  -3.786173  1.710431e-04   

                                  [0.025    0.975]  
const                           0.060675  0.080015  
data analytics course_mobile   -0.035406 -0.008284  
data analytics training_tablet -0.040030 -0.012680  


In [279]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='clicks_share', 
                                      baseline='learn data analytics_mobile')

                                   Coef.  Std.Err.          t         P>|t|  \
const                           0.067428  0.005023  13.424400  1.951519e-35   
data analytics training_tablet -0.022817  0.007103  -3.212221  1.399659e-03   

                                  [0.025    0.975]  
const                           0.057560  0.077295  
data analytics training_tablet -0.036772 -0.008862  


In [280]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='impressions_share', 
                                      baseline='learn data analytics_mobile')

                                   Coef.  Std.Err.          t         P>|t|  \
const                           0.067490  0.004914  13.734210  8.680376e-37   
data analytics training_tablet -0.022676  0.006949  -3.262987  1.175743e-03   

                                  [0.025    0.975]  
const                           0.057836  0.077144  
data analytics training_tablet -0.036329 -0.009023  


In [281]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='leads_share', 
                                      baseline='learn data analytics_mobile')

                                   Coef.  Std.Err.          t         P>|t|  \
const                           0.069628  0.004938  14.101026  2.089641e-38   
data analytics training_tablet -0.026634  0.006983  -3.814158  1.532530e-04   

                                  [0.025    0.975]  
const                           0.059927  0.079328  
data analytics training_tablet -0.040353 -0.012916  


In [282]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='conversions_share', 
                                      baseline='learn data analytics_mobile')

                                   Coef.  Std.Err.          t         P>|t|  \
const                           0.073214  0.005010  14.612490  1.079124e-40   
analytics for data_tablet      -0.022840  0.007026  -3.250654  1.226859e-03   
data analytics course_mobile   -0.026340  0.007026  -3.748774  1.978731e-04   
data analytics online_mobile   -0.021940  0.007026  -3.122566  1.893810e-03   
data analytics online_tablet   -0.023669  0.007086  -3.340378  8.974483e-04   
data analytics training_tablet -0.029800  0.007086  -4.205645  3.071474e-05   
learn data analytics_tablet    -0.021850  0.007026  -3.109757  1.976274e-03   

                                  [0.025    0.975]  
const                           0.063370  0.083057  
analytics for data_tablet      -0.036644 -0.009036  
data analytics course_mobile   -0.040144 -0.012536  
data analytics online_mobile   -0.035744 -0.008136  
data analytics online_tablet   -0.037589 -0.009748  
data analytics training_tablet -0.043721 -0.01587

In [283]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='sales_share', 
                                      baseline='learn data analytics_mobile')

                                   Coef.  Std.Err.          t         P>|t|  \
const                           0.069572  0.005003  13.907179  1.505632e-37   
data analytics online_mobile   -0.021919  0.007016  -3.124345  1.882614e-03   
data analytics training_tablet -0.025000  0.007075  -3.533678  4.467772e-04   

                                  [0.025    0.975]  
const                           0.059744  0.079401  
data analytics online_mobile   -0.035702 -0.008136  
data analytics training_tablet -0.038899 -0.011101  


In [284]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='spend_share', 
                                      baseline='data analytics training_tablet')

                                Coef.  Std.Err.         t         P>|t|  \
const                        0.043990  0.004922  8.937173  7.159727e-18   
learn data analytics_mobile  0.026355  0.006961  3.786173  1.710431e-04   

                              [0.025   0.975]  
const                        0.03432  0.05366  
learn data analytics_mobile  0.01268  0.04003  


In [285]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='clicks_share', 
                                      baseline='data analytics training_tablet')

                                Coef.  Std.Err.         t         P>|t|  \
const                        0.044610  0.005023  8.881634  1.104912e-17   
learn data analytics_mobile  0.022817  0.007103  3.212221  1.399659e-03   

                               [0.025    0.975]  
const                        0.034743  0.054478  
learn data analytics_mobile  0.008862  0.036772  


In [286]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='impressions_share', 
                                      baseline='data analytics training_tablet')

                                Coef.  Std.Err.         t         P>|t|  \
const                        0.044814  0.004914  9.119650  1.698136e-18   
learn data analytics_mobile  0.022676  0.006949  3.262987  1.175743e-03   

                               [0.025    0.975]  
const                        0.035160  0.054468  
learn data analytics_mobile  0.009023  0.036329  


In [287]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='leads_share', 
                                      baseline='data analytics training_tablet')

                                Coef.  Std.Err.         t         P>|t|  \
const                        0.042993  0.004938  8.706992  4.269565e-17   
learn data analytics_mobile  0.026634  0.006983  3.814158  1.532530e-04   

                               [0.025    0.975]  
const                        0.033292  0.052694  
learn data analytics_mobile  0.012916  0.040353  


In [288]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='conversions_share', 
                                      baseline='data analytics training_tablet')

                                Coef.  Std.Err.         t         P>|t|  \
const                        0.043414  0.005010  8.664810  5.901006e-17   
learn data analytics_mobile  0.029800  0.007086  4.205645  3.071474e-05   

                               [0.025    0.975]  
const                        0.033570  0.053257  
learn data analytics_mobile  0.015879  0.043721  


In [289]:
model = run_keyword_device_regression(keyw_dev_shares, keyword_col='Keyword', device_col='Device', target_col='sales_share', 
                                      baseline='data analytics training_tablet')

                                Coef.  Std.Err.         t         P>|t|  \
const                        0.044572  0.005003  8.909803  8.868675e-18   
learn data analytics_mobile  0.025000  0.007075  3.533678  4.467772e-04   

                               [0.025    0.975]  
const                        0.034744  0.054401  
learn data analytics_mobile  0.011101  0.038899  


In [296]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='spend_share', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.140
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.325
Time:                        19:43:27   Log-Likelihood:                 193.36
No. Observations:                  88   AIC:                            -380.7
Df Residuals:                      85   BIC:                            -373.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [297]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='clicks_share', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.8759
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.420
Time:                        19:43:29   Log-Likelihood:                 189.88
No. Observations:                  88   AIC:                            -373.8
Df Residuals:                      85   BIC:                            -366.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [298]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='impressions_share', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.8156
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.446
Time:                        19:43:30   Log-Likelihood:                 192.62
No. Observations:                  88   AIC:                            -379.2
Df Residuals:                      85   BIC:                            -371.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [299]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='leads_share', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7884
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.458
Time:                        19:43:30   Log-Likelihood:                 198.53
No. Observations:                  88   AIC:                            -391.1
Df Residuals:                      85   BIC:                            -383.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [300]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='conversions_share', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7978
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.454
Time:                        19:43:31   Log-Likelihood:                 187.31
No. Observations:                  88   AIC:                            -368.6
Df Residuals:                      85   BIC:                            -361.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [301]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='sales_share', baseline='tablet')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.8435
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.434
Time:                        19:43:32   Log-Likelihood:                 189.41
No. Observations:                  88   AIC:                            -372.8
Df Residuals:                      85   BIC:                            -365.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [302]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='spend_share', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.140
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.325
Time:                        19:43:33   Log-Likelihood:                 193.36
No. Observations:                  88   AIC:                            -380.7
Df Residuals:                      85   BIC:                            -373.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [303]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='clicks_share', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.8759
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.420
Time:                        19:43:35   Log-Likelihood:                 189.88
No. Observations:                  88   AIC:                            -373.8
Df Residuals:                      85   BIC:                            -366.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [304]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='impressions_share', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.8156
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.446
Time:                        19:43:36   Log-Likelihood:                 192.62
No. Observations:                  88   AIC:                            -379.2
Df Residuals:                      85   BIC:                            -371.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [305]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='leads_share', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7884
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.458
Time:                        19:43:37   Log-Likelihood:                 198.53
No. Observations:                  88   AIC:                            -391.1
Df Residuals:                      85   BIC:                            -383.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [307]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='conversions_share', baseline='desktop')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7978
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.454
Time:                        19:43:46   Log-Likelihood:                 187.31
No. Observations:                  88   AIC:                            -368.6
Df Residuals:                      85   BIC:                            -361.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [308]:
model = run_subset_regression(keyw_dev_shares, keyword='analytics for data', target_column='sales_share', baseline='mobile')


--- Regression Summary for Keyword: 'analytics for data' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.8435
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.434
Time:                        19:43:48   Log-Likelihood:                 189.41
No. Observations:                  88   AIC:                            -372.8
Df Residuals:                      85   BIC:                            -365.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [309]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='spend_share', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     1.766
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.177
Time:                        19:43:49   Log-Likelihood:                 202.45
No. Observations:                  90   AIC:                            -398.9
Df Residuals:                      87   BIC:                            -391.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [310]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='clicks_share', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.351
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.101
Time:                        19:43:51   Log-Likelihood:                 203.31
No. Observations:                  90   AIC:                            -400.6
Df Residuals:                      87   BIC:                            -393.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [311]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='impressions_share', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.531
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.222
Time:                        19:44:56   Log-Likelihood:                 202.74
No. Observations:                  90   AIC:                            -399.5
Df Residuals:                      87   BIC:                            -392.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [312]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='leads_share', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.678
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.193
Time:                        19:44:58   Log-Likelihood:                 200.80
No. Observations:                  90   AIC:                            -395.6
Df Residuals:                      87   BIC:                            -388.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [313]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='conversions_share', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     3.160
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0473
Time:                        19:44:59   Log-Likelihood:                 203.33
No. Observations:                  90   AIC:                            -400.7
Df Residuals:                      87   BIC:                            -393.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [314]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='sales_share', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     2.179
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.119
Time:                        19:45:27   Log-Likelihood:                 203.77
No. Observations:                  90   AIC:                            -401.5
Df Residuals:                      87   BIC:                            -394.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [316]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='spend_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     1.766
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.177
Time:                        19:46:03   Log-Likelihood:                 202.45
No. Observations:                  90   AIC:                            -398.9
Df Residuals:                      87   BIC:                            -391.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [318]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='clicks_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.351
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.101
Time:                        19:46:14   Log-Likelihood:                 203.31
No. Observations:                  90   AIC:                            -400.6
Df Residuals:                      87   BIC:                            -393.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [327]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='impressions_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.531
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.222
Time:                        19:47:50   Log-Likelihood:                 202.74
No. Observations:                  90   AIC:                            -399.5
Df Residuals:                      87   BIC:                            -392.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [320]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='leads_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.678
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.193
Time:                        19:47:10   Log-Likelihood:                 200.80
No. Observations:                  90   AIC:                            -395.6
Df Residuals:                      87   BIC:                            -388.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [326]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='conversions_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     3.160
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0473
Time:                        19:47:43   Log-Likelihood:                 203.33
No. Observations:                  90   AIC:                            -400.7
Df Residuals:                      87   BIC:                            -393.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [325]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics course', target_column='sales_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics course' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     2.179
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.119
Time:                        19:47:31   Log-Likelihood:                 203.77
No. Observations:                  90   AIC:                            -401.5
Df Residuals:                      87   BIC:                            -394.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [323]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='spend_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.5307
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.590
Time:                        19:47:14   Log-Likelihood:                 204.50
No. Observations:                  89   AIC:                            -403.0
Df Residuals:                      86   BIC:                            -395.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [328]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='clicks_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.4764
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.623
Time:                        19:48:55   Log-Likelihood:                 205.83
No. Observations:                  89   AIC:                            -405.7
Df Residuals:                      86   BIC:                            -398.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [329]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='impressions_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.1807
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.835
Time:                        19:48:57   Log-Likelihood:                 207.00
No. Observations:                  89   AIC:                            -408.0
Df Residuals:                      86   BIC:                            -400.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [331]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='leads_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9593
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.387
Time:                        19:49:09   Log-Likelihood:                 201.73
No. Observations:                  89   AIC:                            -397.5
Df Residuals:                      86   BIC:                            -390.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [332]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='conversions_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3733
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.690
Time:                        19:49:11   Log-Likelihood:                 206.03
No. Observations:                  89   AIC:                            -406.1
Df Residuals:                      86   BIC:                            -398.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [334]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='sales_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9203
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.402
Time:                        19:49:21   Log-Likelihood:                 207.14
No. Observations:                  89   AIC:                            -408.3
Df Residuals:                      86   BIC:                            -400.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [336]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='spend_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.5307
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.590
Time:                        19:49:29   Log-Likelihood:                 204.50
No. Observations:                  89   AIC:                            -403.0
Df Residuals:                      86   BIC:                            -395.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [338]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='clicks_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.4764
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.623
Time:                        19:49:39   Log-Likelihood:                 205.83
No. Observations:                  89   AIC:                            -405.7
Df Residuals:                      86   BIC:                            -398.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [340]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='impressions_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.1807
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.835
Time:                        19:49:53   Log-Likelihood:                 207.00
No. Observations:                  89   AIC:                            -408.0
Df Residuals:                      86   BIC:                            -400.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [341]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='leads_share', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9593
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.387
Time:                        19:49:56   Log-Likelihood:                 201.73
No. Observations:                  89   AIC:                            -397.5
Df Residuals:                      86   BIC:                            -390.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [343]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='conversions_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3733
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.690
Time:                        19:50:04   Log-Likelihood:                 206.03
No. Observations:                  89   AIC:                            -406.1
Df Residuals:                      86   BIC:                            -398.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [344]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics online', target_column='sales_share', baseline='mobile')


--- Regression Summary for Keyword: 'data analytics online' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9203
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.402
Time:                        19:50:07   Log-Likelihood:                 207.14
No. Observations:                  89   AIC:                            -408.3
Df Residuals:                      86   BIC:                            -400.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [346]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='spend_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     2.765
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0687
Time:                        19:50:15   Log-Likelihood:                 187.12
No. Observations:                  87   AIC:                            -368.2
Df Residuals:                      84   BIC:                            -360.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [348]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='clicks_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     2.423
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0948
Time:                        19:50:58   Log-Likelihood:                 183.65
No. Observations:                  87   AIC:                            -361.3
Df Residuals:                      84   BIC:                            -353.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [350]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='impressions_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     2.867
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0624
Time:                        19:51:08   Log-Likelihood:                 185.68
No. Observations:                  87   AIC:                            -365.4
Df Residuals:                      84   BIC:                            -358.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [352]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='leads_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     2.532
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0856
Time:                        19:51:37   Log-Likelihood:                 184.70
No. Observations:                  87   AIC:                            -363.4
Df Residuals:                      84   BIC:                            -356.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [354]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='conversions_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     4.058
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0208
Time:                        19:52:08   Log-Likelihood:                 186.94
No. Observations:                  87   AIC:                            -367.9
Df Residuals:                      84   BIC:                            -360.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [356]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='sales_share', baseline='desktop')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     2.673
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0749
Time:                        19:52:42   Log-Likelihood:                 184.90
No. Observations:                  87   AIC:                            -363.8
Df Residuals:                      84   BIC:                            -356.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [357]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='spend_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     2.765
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0687
Time:                        19:53:11   Log-Likelihood:                 187.12
No. Observations:                  87   AIC:                            -368.2
Df Residuals:                      84   BIC:                            -360.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [358]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='clicks_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     2.423
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0948
Time:                        19:53:36   Log-Likelihood:                 183.65
No. Observations:                  87   AIC:                            -361.3
Df Residuals:                      84   BIC:                            -353.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [359]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='impressions_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     2.867
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0624
Time:                        19:53:38   Log-Likelihood:                 185.68
No. Observations:                  87   AIC:                            -365.4
Df Residuals:                      84   BIC:                            -358.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [360]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='leads_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     2.532
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0856
Time:                        19:53:59   Log-Likelihood:                 184.70
No. Observations:                  87   AIC:                            -363.4
Df Residuals:                      84   BIC:                            -356.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [361]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='conversions_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     4.058
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0208
Time:                        19:54:18   Log-Likelihood:                 186.94
No. Observations:                  87   AIC:                            -367.9
Df Residuals:                      84   BIC:                            -360.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [362]:
model = run_subset_regression(keyw_dev_shares, keyword='data analytics training', target_column='sales_share', baseline='tablet')


--- Regression Summary for Keyword: 'data analytics training' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     2.673
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0749
Time:                        19:54:50   Log-Likelihood:                 184.90
No. Observations:                  87   AIC:                            -363.8
Df Residuals:                      84   BIC:                            -356.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [363]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='spend_share', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     3.727
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0281
Time:                        19:55:08   Log-Likelihood:                 203.67
No. Observations:                  88   AIC:                            -401.3
Df Residuals:                      85   BIC:                            -393.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [364]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='clicks_share', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.814
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0656
Time:                        19:56:14   Log-Likelihood:                 198.46
No. Observations:                  88   AIC:                            -390.9
Df Residuals:                      85   BIC:                            -383.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [365]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='impressions_share', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     3.121
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0492
Time:                        19:56:34   Log-Likelihood:                 204.61
No. Observations:                  88   AIC:                            -403.2
Df Residuals:                      85   BIC:                            -395.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [366]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='leads_share', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     4.004
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0218
Time:                        19:56:49   Log-Likelihood:                 203.99
No. Observations:                  88   AIC:                            -402.0
Df Residuals:                      85   BIC:                            -394.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [367]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='conversions_share', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     6.692
Date:                Mon, 15 Sep 2025   Prob (F-statistic):            0.00200
Time:                        19:57:08   Log-Likelihood:                 200.68
No. Observations:                  88   AIC:                            -395.4
Df Residuals:                      85   BIC:                            -387.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [368]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='sales_share', baseline='tablet')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     3.013
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0544
Time:                        19:57:34   Log-Likelihood:                 198.50
No. Observations:                  88   AIC:                            -391.0
Df Residuals:                      85   BIC:                            -383.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [369]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='spend_share', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     3.727
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0281
Time:                        19:57:56   Log-Likelihood:                 203.67
No. Observations:                  88   AIC:                            -401.3
Df Residuals:                      85   BIC:                            -393.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [370]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='clicks_share', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.814
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0656
Time:                        19:58:25   Log-Likelihood:                 198.46
No. Observations:                  88   AIC:                            -390.9
Df Residuals:                      85   BIC:                            -383.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [371]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='impressions_share', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     3.121
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0492
Time:                        19:58:41   Log-Likelihood:                 204.61
No. Observations:                  88   AIC:                            -403.2
Df Residuals:                      85   BIC:                            -395.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [372]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='leads_share', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     4.004
Date:                Mon, 15 Sep 2025   Prob (F-statistic):             0.0218
Time:                        19:58:57   Log-Likelihood:                 203.99
No. Observations:                  88   AIC:                            -402.0
Df Residuals:                      85   BIC:                            -394.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [373]:
model = run_subset_regression(keyw_dev_shares, keyword='learn data analytics', target_column='conversions_share', baseline='mobile')


--- Regression Summary for Keyword: 'learn data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     6.692
Date:                Mon, 15 Sep 2025   Prob (F-statistic):            0.00200
Time:                        19:59:11   Log-Likelihood:                 200.68
No. Observations:                  88   AIC:                            -395.4
Df Residuals:                      85   BIC:                            -387.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [375]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='spend_share', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.7330
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.483
Time:                        20:00:06   Log-Likelihood:                 196.44
No. Observations:                  90   AIC:                            -386.9
Df Residuals:                      87   BIC:                            -379.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [376]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='clicks_share', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.436
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.244
Time:                        20:00:11   Log-Likelihood:                 196.24
No. Observations:                  90   AIC:                            -386.5
Df Residuals:                      87   BIC:                            -379.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [377]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='impressions_share', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.4042
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.669
Time:                        20:00:13   Log-Likelihood:                 196.62
No. Observations:                  90   AIC:                            -387.2
Df Residuals:                      87   BIC:                            -379.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [378]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='leads_share', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.8470
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.432
Time:                        20:00:19   Log-Likelihood:                 196.29
No. Observations:                  90   AIC:                            -386.6
Df Residuals:                      87   BIC:                            -379.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [379]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='conversions_share', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.041
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.357
Time:                        20:00:21   Log-Likelihood:                 194.61
No. Observations:                  90   AIC:                            -383.2
Df Residuals:                      87   BIC:                            -375.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [380]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='sales_share', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.572
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.213
Time:                        20:00:24   Log-Likelihood:                 195.91
No. Observations:                  90   AIC:                            -385.8
Df Residuals:                      87   BIC:                            -378.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [381]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='spend_share', baseline='mobile')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            spend_share   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.7330
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.483
Time:                        20:00:26   Log-Likelihood:                 196.44
No. Observations:                  90   AIC:                            -386.9
Df Residuals:                      87   BIC:                            -379.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [383]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='clicks_share', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:           clicks_share   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.436
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.244
Time:                        20:00:34   Log-Likelihood:                 196.24
No. Observations:                  90   AIC:                            -386.5
Df Residuals:                      87   BIC:                            -379.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [385]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='impressions_share', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      impressions_share   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.4042
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.669
Time:                        20:00:44   Log-Likelihood:                 196.62
No. Observations:                  90   AIC:                            -387.2
Df Residuals:                      87   BIC:                            -379.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [387]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='leads_share', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            leads_share   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.8470
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.432
Time:                        20:00:52   Log-Likelihood:                 196.29
No. Observations:                  90   AIC:                            -386.6
Df Residuals:                      87   BIC:                            -379.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [389]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='conversions_share', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:      conversions_share   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.041
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.357
Time:                        20:01:00   Log-Likelihood:                 194.61
No. Observations:                  90   AIC:                            -383.2
Df Residuals:                      87   BIC:                            -375.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [391]:
model = run_subset_regression(keyw_dev_shares, keyword='online data analytics', target_column='sales_share', baseline='desktop')


--- Regression Summary for Keyword: 'online data analytics' ---

                            OLS Regression Results                            
Dep. Variable:            sales_share   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.572
Date:                Mon, 15 Sep 2025   Prob (F-statistic):              0.213
Time:                        20:01:10   Log-Likelihood:                 195.91
No. Observations:                  90   AIC:                            -385.8
Df Residuals:                      87   BIC:                            -378.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------